In [ ]:
import sys
sys.path.insert(0, "..")
from src.config import PROJECT_ROOT, MODEL_SEED, LABELS
from src.variables import LLM_COLS, LAB_COLS, CODE_COLS, CATEGORY_COLS
from src.feature_utils import (
    find_feature_list_file, read_feature_list, load_xy,
    load_feature_name_map, prettify_name, prettify_names,
    clean_ct_feature_name, inject_psych_scale_aliases, prettify_psych_name,
    FRIENDLY_OVERRIDES, LEVEL_MAP, to_bayes_space
)
from src.models import get_models_and_search_space
from src.preprocessing import create_preprocessor
from src.evaluation import youden_threshold, metrics_at_threshold, decision_curve

import os, re, joblib, json, glob, warnings
import numpy as np
import pandas as pd
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Tuple
from itertools import combinations

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss,
    precision_recall_fscore_support, accuracy_score,
    roc_curve, confusion_matrix,
)
from sklearn.calibration import calibration_curve
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin, clone

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

import shap
import matplotlib.pyplot as plt
from upsetplot import from_contents, plot
from sklearn.manifold import TSNE
import umap.umap_ as umap

warnings.filterwarnings("ignore", category=UserWarning)
plt.rcParams.update(
    {"figure.dpi": 130, "axes.spines.top": False, "axes.spines.right": False}
)


In [ ]:
# ---------------------------
# 0) 경로/조합 설정
# ---------------------------
OUT_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20")
)
FIG_DIR = OUT_DIR / "figures"
OUT_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

IMP_DIR = Path(
    str(PROJECT_ROOT / "data/processed_imp/260114_split_corr_LLM_ADER/imputation/simple_imput")
)
FS_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/Feature_Selection/simple_20/step2_FS")
)

# Train/Test 파일 이름
DATA_FILES = {
    "label_30d": {
        "train": IMP_DIR / "simple_label_30d_train.csv",
        "test": IMP_DIR / "simple_label_30d_test.csv",
    },
    "label_60d": {
        "train": IMP_DIR / "simple_label_60d_train.csv",
        "test": IMP_DIR / "simple_label_60d_test.csv",
    },
    "label_90d": {
        "train": IMP_DIR / "simple_label_90d_train.csv",
        "test": IMP_DIR / "simple_label_90d_test.csv",
    },
    "label_180d": {
        "train": IMP_DIR / "simple_label_180d_train.csv",
        "test": IMP_DIR / "simple_label_180d_test.csv",
    },
    "label_365d": {
        "train": IMP_DIR / "simple_label_365d_train.csv",
        "test": IMP_DIR / "simple_label_365d_test.csv",
    },
}

# 당신의 탐색 결과를 반영한 권장조합 (필요시 수정)
BEST_COMBOS = {
    "label_30d": {"model": "rf", "feature_set": "All_Features"},
    "label_60d": {"model": "rf", "feature_set": "All_Features"},
    "label_90d": {"model": "rf", "feature_set": "All_Features"},
    "label_180d": {"model": "rf", "feature_set": "All_Features"},
    "label_365d": {"model": "rf", "feature_set": "All_Features"},
}

TARGET_COL_FALLBACK = "label"

In [ ]:
FEATURE_MAP_CSV = str(PROJECT_ROOT / "results/new_analysis/260106/Feature Selection/simple_20/feature_summary.csv")
NAME_MAP = load_feature_name_map(FEATURE_MAP_CSV)

## UMAP + DBSCAN

## ⭐️⭐️⭐️ version 3 ⭐️⭐️⭐️

In [ ]:
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from scipy.stats import chi2_contingency, kruskal
from scipy.spatial import ConvexHull
import matplotlib.patches as mpatches
import matplotlib.cm as cm
from adjustText import adjust_text

try:
    import umap.umap_ as UMAP

    HAS_UMAP = True
except Exception:
    HAS_UMAP = False

In [ ]:
# =============== 경로/전역 ===============
MODEL_DIR = str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20")
OUT_ROOT = Path(MODEL_DIR) / "figures/clinic_interpretation/PatientClusters"
OUT_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
LLM_ALIAS = {
    "Impaired_Social_Function": "Social Function Impairment (LLM)",
    "Religious_Affiliation": "Religious Affiliation (LLM)",
    "Violence_and_Impulsivity": "Aggression/Impulsivity (LLM)",
    "Domestic_Violence": "Domestic Violence (LLM)",
    "Physical_Abuse": "Physical Abuse (LLM)",
    "Divorce": "Divorce Experience (LLM)",
    "Death_of_Family_Member": "Family Loss (LLM)",
    "Emotional_Abuse": "Emotional Abuse (LLM)",
    "Lack_of_Family_Support": "Lack of Family Support (LLM)",
    "Social_Isolation_and_Lack_of_Support": "Social Isolation (LLM)",
    "Psychotic_Symptoms": "Halluc/Delusion/Paranoia (LLM)",
    "Interpersonal_Conflict": "Interpersonal Conflict (LLM)",
    "Exposure_to_Suicide": "Suicide Exposure (LLM)",
    "Alcohol_Use_Problems": "Alcohol Use Issues (LLM)",
    "Sexual_Abuse": "Sexual Victimization (LLM)",
    "Physical_and_Emotional_Neglect": "Neglect (LLM)",
}

def harmonize_columns_for_pipeline(X, pipe):
    """
    파이프라인의 전처리기가 기대하는 컬럼 구성과 순서로 데이터프레임을 재정렬하는 함수
    """
    # 보통 파이프라인의 첫 번째 스텝(전처리기)에서 피처 이름을 가져옵니다.
    try:
        expected_cols = pipe.named_steps["preprocessor"].feature_names_in_
    except AttributeError:
        # 피처 이름이 저장되지 않은 경우 등에 대한 예외 처리
        return X

    # 누락된 컬럼은 0으로 채우고, 순서를 맞춤
    for col in expected_cols:
        if col not in X.columns:
            X[col] = 0

    return X[expected_cols]

def harmonize_with_alias(X, pipe, alias_map=LLM_ALIAS, fill_value=0):
    X = X.copy()

    # 1) alias 컬럼이 있으면, 파이프라인이 기대하는 "원본키" 컬럼로 복사 생성
    for raw_key, pretty_name in alias_map.items():
        if (raw_key not in X.columns) and (pretty_name in X.columns):
            X[raw_key] = X[pretty_name]

    # 2) 기존 harmonize (없는 컬럼 생성/정렬 등)
    X = harmonize_columns_for_pipeline(X, pipe)

    return X

def shap_on_test(label: str, max_bg=200, force_tree_interventional=True):
    pipe = load_best_pipeline(label)
    preproc = pipe.named_steps["preprocessor"]
    clf = pipe.named_steps["clf"]

    # ✅ 이거 반드시 필요
    X_raw, y_true, df_pred = load_train_test(label)

    # ✅ alias -> raw_key 복구 후 harmonize
    X_raw_h = harmonize_with_alias(X_raw, pipe)

    X_te_t = preproc.transform(X_raw_h)
    feat_names = load_feature_names_transformed(label)

    rng = np.random.RandomState(42)
    bg_idx = rng.choice(X_te_t.shape[0], size=min(max_bg, X_te_t.shape[0]), replace=False)
    X_bg_t = X_te_t[bg_idx]

    try:
        if force_tree_interventional:
            explainer = shap.TreeExplainer(
                clf,
                data=X_bg_t,
                feature_perturbation="interventional",
                model_output="probability",
            )
        else:
            explainer = shap.Explainer(clf, X_bg_t)
        expl = explainer(X_te_t)
    except Exception:
        expl = shap.Explainer(clf, X_bg_t)(X_te_t)

    vals = np.array(expl.values)
    if vals.ndim == 3 and vals.shape[2] == 2:
        expl = expl[:, :, 1]

    try:
        expl.feature_names = feat_names
    except Exception:
        pass

    return expl, X_te_t, y_true, df_pred

In [ ]:
def _standardize(X: np.ndarray) -> np.ndarray:
    X = np.asarray(X, dtype=float)
    X = np.nan_to_num(
        X, nan=0.0, posinf=np.finfo(float).max / 1e6, neginf=-np.finfo(float).max / 1e6
    )
    return (X - X.mean(0)) / (X.std(0) + 1e-9)


def save_png_svg(figpath: Path, **savefig_kwargs):
    plt.savefig(figpath, **savefig_kwargs)
    try:
        plt.savefig(
            figpath.with_suffix(".svg"),
            bbox_inches=savefig_kwargs.get("bbox_inches", "tight"),
        )
    except Exception:
        pass
    plt.close()


def reorder_labels_by_risk(labels: np.ndarray, y_true: np.ndarray):
    """
    클러스터 라벨을 '실제 발생률(Event Rate)' 오름차순으로 재정렬합니다.
    - Cluster 0: 발생률이 가장 낮은 그룹
    - Cluster N: 발생률이 가장 높은 그룹
    - Cluster -1: 노이즈 (변경 없음)
    """
    # 노이즈 제외한 유니크 라벨
    unique_labels = [c for c in np.unique(labels) if c != -1]

    if not unique_labels:
        return labels

    # (구 라벨, Event Rate) 리스트 생성
    risk_scores = []
    print("\n[Event Rate Reordering Check]")
    for c in unique_labels:
        # 해당 클러스터의 실제 발생률 계산 (Mean of y_true)
        # y_true가 0/1 이진값이라 가정하면 평균이 곧 발생률입니다.
        event_rate = np.mean(y_true[labels == c])
        risk_scores.append((c, event_rate))
        print(f" - Original Cluster {c}: Event Rate = {event_rate:.4f}")

    # Event Rate 기준 오름차순 정렬 (낮음 -> 높음)
    risk_scores.sort(key=lambda x: x[1])

    # 매핑 딕셔너리 생성
    print("-> Mapping Result:")
    mapping = {-1: -1}
    for new_idx, (old_label, rate) in enumerate(risk_scores):
        mapping[old_label] = new_idx
        print(f"   Old {old_label} (Rate {rate:.4f}) -> New {new_idx} (Low to High)")

    # 새로운 라벨 적용
    new_labels = np.array([mapping[x] for x in labels])

    return new_labels


# =============== Youden 임계값 ===============
from sklearn.metrics import roc_curve


def _try_read_youden_threshold(label: str, base_dir: Path = OUT_ROOT.parent):
    for sep in ["__", "___"]:
        f = base_dir / f"{label}{sep}youden_metrics_Test.csv"
        if f.exists():
            try:
                dfy = pd.read_csv(f)
                if "threshold" in dfy.columns:
                    return float(dfy.loc[0, "threshold"])
            except Exception:
                pass
    return None


def _compute_youden_from_preds(y_true: np.ndarray, y_prob: np.ndarray) -> float:
    fpr, tpr, thr = roc_curve(y_true, y_prob)
    J = tpr - fpr
    return float(thr[int(np.argmax(J))])


# =============== SHAP 로딩 ===============
def get_shap_matrix(label: str):
    """
    Returns:
      expl: shap.Explanation (n, p)
      vals: ndarray (n, p)
      feat_names: list[str] length p (cleaned)
      y_prob: ndarray (n,)
      y_true: ndarray (n,)
      X_te_t: ndarray (n, p_after_preproc)
      df_pred: DataFrame with y_prob[, y_true, y_pred]
    """
    # ⚠️ 기존 프로젝트의 함수 가정
    expl, X_te_t, y_true, df_pred = shap_on_test(label)

    vals = np.asarray(expl.values, dtype=float)
    vals = np.nan_to_num(
        vals,
        nan=0.0,
        posinf=np.finfo(float).max / 1e6,
        neginf=-np.finfo(float).max / 1e6,
    )

    if getattr(expl, "feature_names", None) is not None:
        raw_names = list(expl.feature_names)
    else:
        raw_names = [f"f{i}" for i in range(vals.shape[1])]
    feat_names = [clean_name(n) for n in raw_names]

    y_prob = np.asarray(df_pred["y_prob"].values, dtype=float)
    y_true = np.asarray(getattr(y_true, "values", y_true)).astype(int)
    return expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred


# =============== k-distance & eps 추정 ===============
def _kdist_eps_heuristic(X_latent: np.ndarray, k: int = 5, pct: float = 0.85) -> float:
    nn = NearestNeighbors(n_neighbors=max(k, 2), metric="euclidean")
    nn.fit(X_latent)
    dists, _ = nn.kneighbors(X_latent)
    kth = np.sort(dists[:, -1])
    base_eps = float(np.quantile(kth, pct))
    return base_eps, kth


def save_kdist_plot(kth_sorted: np.ndarray, k: int, out_path: Path):
    plt.figure(figsize=(4, 3))
    plt.plot(np.arange(len(kth_sorted)), kth_sorted)
    plt.xlabel("Points (sorted)")
    plt.ylabel(f"{k}-NN distance")
    plt.title(f"k-distance curve (k={k})")
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")


# =============== 히트맵 ===============
def _agg_abs(values: np.ndarray, how="mean"):
    return (
        np.abs(values).mean(axis=0)
        if how == "mean"
        else np.mean(np.abs(values), axis=0)
    )


def plot_cluster_heatmap(
    vals,
    feat_names,
    labels,
    topN,
    out_path: Path,
    cmap: str = "rocket",
    agg: str = "mean",
):
    """
    반환:
      {
        "raw_all":  DataFrame (clusters × all_features) : 클러스터별 원시 집계 |SHAP|
        "raw_top":  DataFrame (clusters × topN)         : 히트맵에 사용된 TopN의 원시 |SHAP|
        "norm_top": DataFrame (clusters × topN)         : TopN의 행 정규화(상대값)
        "feat_top": List[str]                           : TopN 피처명
        "clusters": np.ndarray                          : 사용된 클러스터 ID (노이즈 -1 제외)
        "fig_raw_path": Path                            : raw heatmap PNG 경로
        "fig_norm_path": Path                           : normalized heatmap PNG 경로(= out_path)
      }
    """
    from textwrap import fill

    # 노이즈(-1) 제외
    clusters = [c for c in sorted(np.unique(labels)) if c != -1]
    if not clusters:
        return None

    # --- (A) 클러스터별 원시 집계 |SHAP|
    rows = []
    for c in clusters:
        mask = labels == c
        rows.append(_agg_abs(vals[mask], how=agg))  # shape: (p,)
    M = np.vstack(rows)  # (n_clusters, n_features)

    # --- (B) 히트맵에서 쓸 TopN 선택 (클러스터 max 기준)
    global_imp = _agg_abs(vals, how=agg)  # 전반적 중요도 (median|SHAP| 또는 mean|SHAP|)
    hetero = M.max(axis=0) - M.min(axis=0)  # 간단한 이질성 점수 (max - min)

    topK_imp = max(topN * 2, 30)  # 후보 풀(전반적 중요도 상위 K)
    cand = np.argsort(global_imp)[::-1][:topK_imp]
    top_idx = cand[
        np.argsort(hetero[cand])[::-1][:topN]
    ]  # 후보 안에서 이질성 큰 순 TopN

    M_top = M[:, top_idx]
    feat_top = [feat_names[i] for i in top_idx]

    ###########################################
    # global_imp = _agg_abs(vals, how=agg)  # shape (p,)  # median(|SHAP|)
    # top_idx = np.argsort(global_imp)[::-1][:topN]

    # M_top = M[:, top_idx]
    # feat_top = [feat_names[i] for i in top_idx]
    ###########################################
    # top_idx = np.argsort(M.max(axis=0))[::-1][:topN]
    # M_top = M[:, top_idx]  # raw topN
    # feat_top = [feat_names[i] for i in top_idx]

    # --- (C) 행 정규화(클러스터 내부 상대값)
    row_max = np.maximum(M_top.max(axis=1, keepdims=True), 1e-9)
    M_disp = M_top / row_max  # normalized topN

    # --- (D) DataFrame 구성
    idx_names = [f"cluster_{c}" for c in clusters]
    df_raw_all = pd.DataFrame(M, index=idx_names, columns=feat_names)
    df_raw_top = pd.DataFrame(M_top, index=idx_names, columns=feat_top)
    df_norm_top = pd.DataFrame(M_disp, index=idx_names, columns=feat_top)

    # ---------- (E) Normalized Heatmap (기존 파일명 유지) ----------
    xticks = [f"Cluster {c}" for c in clusters]
    yticks = [fill(f, width=28) for f in feat_top]
    fig_w = max(5, 0.8 * len(clusters))
    fig_h = max(4, 0.35 * len(yticks))

    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        df_norm_top.T.values,
        cmap=cmap,
        cbar_kws={"label": "Relative importance"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )
    ax.set_title(
        f"Cluster × Feature importance ({agg} |SHAP|)",  #  – Row-normalized
        fontsize=12,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features", fontsize=10, labelpad=6)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    ax.tick_params(axis="y", labelsize=8, pad=4)
    ax.collections[0].colorbar.ax.tick_params(labelsize=9)
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")

    # ---------- (F) Raw Heatmap (새 파일) ----------
    out_path_raw = out_path.with_name(out_path.stem + "__RAW.png")
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        df_raw_top.T.values,
        cmap=cmap,
        cbar_kws={"label": f"{agg} |SHAP| (raw)"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )
    ax.set_title(
        f"Cluster × Feature importance ({agg} |SHAP|) – Raw",
        fontsize=12,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features", fontsize=10, labelpad=6)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    ax.tick_params(axis="y", labelsize=8, pad=4)
    ax.collections[0].colorbar.ax.tick_params(labelsize=9)
    plt.tight_layout()
    save_png_svg(out_path_raw, dpi=300, bbox_inches="tight")

    return {
        "raw_all": df_raw_all,
        "raw_top": df_raw_top,
        "norm_top": df_norm_top,
        "feat_top": feat_top,
        "clusters": np.array(clusters),
        "fig_raw_path": out_path_raw,
        "fig_norm_path": out_path,
    }


# =============== 대표 케이스 ===============
def pick_representatives(y_prob: np.ndarray, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:  # 노이즈 제외(원하면 포함 가능)
            continue
        idx = np.where(labels == c)[0]
        probs = y_prob[idx]
        med = np.mean(probs)
        med_idx = idx[np.argmin(np.abs(probs - med))]
        max_idx = idx[np.argmax(probs)]
        min_idx = idx[np.argmin(probs)]
        reps.append(
            {
                "cluster": int(c),
                "mean_idx": int(med_idx),
                "max_idx": int(max_idx),
                "min_idx": int(min_idx),
            }
        )
    return reps


def pick_error_representatives(df_pred: pd.DataFrame, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:
            continue
        idx = np.where(labels == c)[0]
        d = df_pred.iloc[idx]
        rec = {"cluster": int(c)}
        fp = d[(d["y_true"] == 0) & (d["y_pred"] == 1)]
        fn = d[(d["y_true"] == 1) & (d["y_pred"] == 0)]
        if len(fp):
            rec["fp_idx"] = int(fp["y_prob"].idxmax())
        if len(fn):
            rec["fn_idx"] = int(fn["y_prob"].idxmin())
        reps.append(rec)
    return reps


def save_local_plots(
    label: str,
    expl: shap.Explanation,
    idx: int,
    out_dir: Path,
    feat_names=None,
    max_display=15,
    df_pred: pd.DataFrame | None = None,
    youden_thr: float | None = None,
    name_tag: str | None = None,
):
    out_dir.mkdir(parents=True, exist_ok=True)
    y_t = y_p = None
    y_prob = None
    thr_txt = ""
    if df_pred is not None:
        if "y_true" in df_pred.columns:
            y_t = int(df_pred.loc[idx, "y_true"])
        if "y_prob" in df_pred.columns:
            y_prob = float(df_pred.loc[idx, "y_prob"])
        if "y_pred" in df_pred.columns:
            y_p = int(df_pred.loc[idx, "y_pred"])
        else:
            thr = youden_thr if youden_thr is not None else 0.5
            y_p = (
                int((df_pred.loc[idx, "y_prob"]) >= thr)
                if "y_prob" in df_pred.columns
                else None
            )
        if youden_thr is not None:
            thr_txt = f", thr={youden_thr:.3f}"

    # waterfall
    plt.figure(figsize=(10, 8))
    try:
        shap.plots.waterfall(expl[idx], show=False, max_display=max_display)
    except Exception:
        row = expl[idx]
        shap.plots.waterfall(row, show=False, max_display=max_display)
    title_bits = [f"{label} | id={idx}"]
    if y_t is not None:
        title_bits.append(f"y_true={y_t}")
    if y_p is not None:
        title_bits.append(f"y_pred={y_p}")
    if y_prob is not None:
        title_bits.append(f"y_prob={y_prob:.3f}")
    if name_tag:
        title_bits.append(f"[{name_tag}]")
    ttl = " , ".join(title_bits) + thr_txt + " (Youden thr on TEST)"
    plt.title(ttl)
    plt.tight_layout()
    save_png_svg(
        out_dir
        / (
            f"{label}__waterfall_id{idx}"
            + (f"__{name_tag}" if name_tag else "")
            + ".png"
        ),
        dpi=300,
        bbox_inches="tight",
    )

    # force (옵션)
    try:
        plt.figure(figsize=(7, 2.8))
        shap.plots.force(
            expl[idx].base_values, expl[idx].values, matplotlib=True, show=False
        )
        plt.tight_layout()
        save_png_svg(
            out_dir
            / (
                f"{label}__force_id{idx}"
                + (f"__{name_tag}" if name_tag else "")
                + ".png"
            ),
            dpi=300,
        )
    except Exception:
        plt.close()


# =============== DBSCAN 통계/요약 ===============
def add_dbscan_stats(
    out_dir: Path,
    label: str,
    y_true: np.ndarray,
    y_prob: np.ndarray,
    labels: np.ndarray,
):
    lab = np.asarray(labels)
    clusters = [c for c in np.unique(lab) if c != -1]
    rows = []
    for c in clusters:
        m = lab == c
        rows.append(
            dict(
                cluster=int(c),
                n=int(m.sum()),
                event_rate=float(np.mean(y_true[m] == 1)),
                mean_y_prob=float(np.mean(y_prob[m])),
            )
        )
    df = pd.DataFrame(rows).sort_values("n", ascending=False)

    chi_p = np.nan
    kw_p = np.nan
    if len(clusters) >= 2:
        table = [
            [
                int(np.sum((lab == c) & (y_true == 1))),
                int(np.sum((lab == c) & (y_true == 0))),
            ]
            for c in clusters
        ]
        chi_p = chi2_contingency(table)[1]
        groups = [y_prob[lab == c] for c in clusters]
        kw_p = kruskal(*groups).pvalue

    df["p_event_rate(chi2)"] = chi_p
    df["p_yprob(kruskal)"] = kw_p
    df.to_csv(out_dir / f"{label}__dbscan_cluster_stats.csv", index=False)
    return df


# =============== 메인 루틴 ===============
def run_cluster_analysis_for_label(
    label: str,
    umap_n_components: int = 12,  # 군집용 UMAP 차원
    umap_n_neighbors: int = 12,
    umap_min_dist: float = 0.01,
    umap_metric: str = "euclidean",  # cosine
    min_samples: int = 6,  # DBSCAN
    eps: float | None = None,
    dbscan_metric: str = "euclidean",
    target_min_clusters: int = 2,
    max_noise_ratio: float = 0.45,
    max_trials: int = 5,
    topN: int = 10,
    agg_for_heatmap: str = "mean",
    save_representatives: bool = True,
    random_state: int = 42,
):
    assert HAS_UMAP, "umap-learn 패키지가 필요합니다."
    print(f"\n=== {label}: UMAP+DBSCAN (adaptive) ===")
    out_dir = OUT_ROOT / label
    out_dir.mkdir(parents=True, exist_ok=True)

    # 0) 데이터 로드
    expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred = get_shap_matrix(label)

    # Youden 임계값 & df_pred 보정
    ythr = _try_read_youden_threshold(label, base_dir=OUT_ROOT.parent)
    if ythr is None:
        ythr = _compute_youden_from_preds(
            np.asarray(y_true).astype(int), np.asarray(y_prob)
        )
    if "y_true" not in df_pred.columns:
        df_pred = df_pred.assign(y_true=np.asarray(y_true).astype(int))
    if "y_pred" not in df_pred.columns and "y_prob" in df_pred.columns:
        df_pred = df_pred.assign(y_pred=(df_pred["y_prob"] >= ythr).astype(int))

    # 1) 표준화 & UMAP(고차원; 군집용)
    # V = _standardize(vals) # 표준화
    V = vals
    um_hi = UMAP.UMAP(
        n_components=umap_n_components,
        n_neighbors=umap_n_neighbors,
        min_dist=umap_min_dist,
        metric=umap_metric,
        random_state=random_state,
    )
    X_hi = um_hi.fit_transform(V)

    # 2) eps 초기값 (k-distance) & 곡선 저장
    if eps is None:
        eps0, kth = _kdist_eps_heuristic(X_hi, k=min_samples, pct=0.9)  # 0.85
        eps = eps0
        save_kdist_plot(
            kth,
            k=min_samples,
            out_path=(out_dir / f"{label}__kdist_k{min_samples}.png"),
        )

    # 3) DBSCAN 적응형 루프
    trial = 0
    labels = None
    info_hist = []
    cur_min_samples = int(min_samples)
    cur_eps = float(eps)

    while trial < max_trials:
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)  # X_hi
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))

        info_hist.append(
            dict(
                trial=trial,
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

        if (n_clusters >= target_min_clusters) and (noise_ratio <= max_noise_ratio):
            break

        # 조정 규칙
        if noise_ratio > max_noise_ratio:
            cur_eps *= 1.20
            if cur_min_samples > 3:
                cur_min_samples -= 1
        elif n_clusters < target_min_clusters:
            cur_eps *= 0.88
        else:
            cur_eps *= 1.05

        trial += 1

    # all-noise 대비: 마지막 완화 시도 1회
    if (labels is None) or np.all(labels == -1):
        cur_eps *= 1.30
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)  # X_hi
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))
        info_hist.append(
            dict(
                trial="final_relax",
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

    # 4) 시각화용 2D UMAP (시각화 전용)
    if umap_n_components == 2:
        Z2 = X_hi
    else:
        um_2d = UMAP.UMAP(
            n_components=2,
            n_neighbors=max(umap_n_neighbors, 15),
            min_dist=max(umap_min_dist, 0.05),
            metric=umap_metric,
            random_state=random_state,
        )
        Z2 = um_2d.fit_transform(V)

    # 5) 라벨 매핑 저장(노이즈 포함)
    m = pd.DataFrame({"idx": np.arange(len(labels)), "cluster": labels})
    m.to_csv(out_dir / f"{label}__dbscan_labels.csv", index=False)

    # 6) 2D 플롯 (클러스터 색)
    uniq = sorted(np.unique(labels))
    cmap = plt.get_cmap("tab10")  # tab20
    color_map = {c: ("#bbbbbb" if c == -1 else cmap(c % 20)) for c in uniq}
    colors = [color_map[c] for c in labels]
    plt.figure(figsize=(7.2, 6.2))
    plt.scatter(Z2[:, 0], Z2[:, 1], c=colors, s=18, alpha=0.9)
    title = f"UMAP + DBSCAN ({umap_n_components}D)"
    plt.title(f"{title} — {len(set(labels)-{-1})} clusters, noise={(labels==-1).sum()}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.margins(0.02)
    save_png_svg(
        out_dir / f"{label}__UMAP2D_DBSCAN_tuned.png", dpi=300, bbox_inches="tight"
    )

    # 6-2) 2D 플롯 (위험도 색)
    plt.figure(figsize=(7.2, 6.2))
    sc = plt.scatter(Z2[:, 0], Z2[:, 1], c=y_prob, s=18, alpha=0.9, cmap="plasma")
    plt.colorbar(sc, label="Predicted risk (prob.)")
    plt.title(f"UMAP(2D) (colored by risk) — {label}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.margins(0.02)
    save_png_svg(
        out_dir / f"{label}__UMAP2D_risk_DBSCAN.png", dpi=300, bbox_inches="tight"
    )

    # 7) 클러스터 통계/요약 + 히트맵
    df_stats = add_dbscan_stats(
        out_dir,
        label,
        y_true=np.asarray(y_true),
        y_prob=np.asarray(y_prob),
        labels=np.asarray(labels),
    )
    ret = plot_cluster_heatmap(
        vals,
        feat_names,
        labels,
        topN=topN,
        out_path=out_dir / f"{label}__heatmap_top{topN}_dbscan.png",
        cmap="magma_r",  # rocket_r, viridis_r
        agg=agg_for_heatmap,
    )

    # === 수치 저장(요청 파일명 규칙) ===
    if ret is not None:
        # 모든 피처의 원시 |SHAP| (클러스터 × 전체 피처)
        ret["raw_all"].to_csv(
            out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__ALL_FEATURES.csv",
            index=True,
        )
        # 히트맵에 쓰인 TopN의 원시 |SHAP|
        ret["raw_top"].to_csv(
            out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_RAW.csv",
            index=True,
        )
        # 히트맵에 쓰인 TopN의 행 정규화(상대값)
        ret["norm_top"].to_csv(
            out_dir
            / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_ROWNORM.csv",
            index=True,
        )

    # 8) 대표 케이스 저장 (mean/max/min + FP/FN)
    if save_representatives:
        rep_dir = out_dir / "representatives_dbscan"
        # mean/max/min
        reps = pick_representatives(y_prob, np.asarray(labels))
        for rep in reps:
            c = rep["cluster"]
            for tag in ["mean_idx", "max_idx", "min_idx"]:
                i = rep[tag]
                name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                save_local_plots(
                    label,
                    expl,
                    i,
                    rep_dir / f"cluster_{c}",
                    feat_names=feat_names,
                    max_display=15,
                    df_pred=df_pred,
                    youden_thr=ythr,
                    name_tag=name_tag,
                )
        # FP/FN
        err_reps = pick_error_representatives(df_pred, np.asarray(labels))
        for rep in err_reps:
            c = rep["cluster"]
            for tag in ["fp_idx", "fn_idx"]:
                if tag in rep:
                    i = rep[tag]
                    name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                    save_local_plots(
                        label,
                        expl,
                        i,
                        rep_dir / f"cluster_{c}",
                        feat_names=feat_names,
                        max_display=15,
                        df_pred=df_pred,
                        youden_thr=ythr,
                        name_tag=name_tag,
                    )

    # 9) 메타(JSON)
    meta = dict(
        timestamp=time.strftime("%Y-%m-%d %H:%M:%S"),
        algo="dbscan_tuned",
        random_state=random_state,
        umap=dict(
            n_components=umap_n_components,
            n_neighbors=umap_n_neighbors,
            min_dist=umap_min_dist,
            metric=umap_metric,
        ),
        final=dict(eps=float(cur_eps), min_samples=int(cur_min_samples)),
        trials=len([h for h in info_hist if isinstance(h.get("trial"), int)]),
        n_clusters=int(len(set(labels) - {-1})),
        noise_ratio=float(np.mean(np.asarray(labels) == -1)),
        history=info_hist,
    )
    with open(out_dir / f"{label}__dbscan_meta.json", "w") as f:
        json.dump(meta, f, indent=2)

    print(f"-> saved: {out_dir}")
    return df_stats, labels

In [ ]:
def plot_umap_for_ppt(
    Z2: np.ndarray,
    labels: np.ndarray,
    y_prob: np.ndarray,
    y_true: np.ndarray,  # [NEW] 실제 정답 레이블 추가
    out_path: Path,
    title: str,
):

    # plt.rcParams.update({'font.size': 14})

    # --- [설정] Risk 판단 기준 (Event Rate 기준) ---
    def get_risk_style(event_rate):
        # Event Rate는 보통 확률보다 낮으므로 기준을 조금 조정할 수도 있음 (현행 유지)
        if event_rate < 0.1:
            return "Low", "#4575b4"
        elif event_rate < 0.3:
            return "Moderate", "#7b3294"
        elif event_rate < 0.5:
            return "High", "#f1a340"
        else:
            return "Very High", "#d73027"

    uniq = sorted(np.unique(labels))
    clusters = [c for c in uniq if c != -1]
    n_clusters = len(clusters)

    global_center = np.mean(Z2, axis=0)
    cluster_cmap = plt.get_cmap("tab20" if n_clusters > 10 else "tab10")

    centroids = {}
    for c in clusters:
        centroids[c] = np.mean(Z2[labels == c], axis=0)

    fig, ax = plt.subplots(figsize=(12, 10))
    ax.scatter(
        Z2[:, 0], Z2[:, 1], c="#e0e0e0", s=20, alpha=0.4, edgecolors="none", zorder=0
    )

    label_items = []
    legend_handles = []

    for i, c in enumerate(clusters):
        mask = labels == c
        points = Z2[mask]

        # [변경] 실제 Event Rate 계산
        # y_true가 0/1로 되어 있다고 가정 (1=Event)
        true_vals = y_true[mask]
        event_rate = np.mean(true_vals == 1)

        risk_name, risk_color = get_risk_style(event_rate)  # Event Rate 기준 스타일
        cluster_color = cluster_cmap(i % 20)

        handle = mpatches.Patch(color=cluster_color, label=f"Cluster {c}")
        legend_handles.append(handle)

        if len(points) < 3:
            ax.scatter(
                points[:, 0], points[:, 1], c=cluster_color, s=20, alpha=0.8, zorder=2
            )
            continue

        ax.scatter(
            points[:, 0],
            points[:, 1],
            c=[cluster_color],
            s=20,
            alpha=0.8,
            edgecolors="white",
            linewidth=0.2,
            zorder=2,
        )
        hull = ConvexHull(points)
        hull_coords = points[np.append(hull.vertices, hull.vertices[0])]
        ax.plot(
            hull_coords[:, 0],
            hull_coords[:, 1],
            color=cluster_color,
            linestyle="--",
            linewidth=1.2,
            alpha=0.8,
            zorder=3,
        )
        ax.add_patch(
            mpatches.Polygon(
                hull_coords,
                closed=True,
                fc=cluster_color,
                ec=None,
                alpha=0.08,
                zorder=1,
            )
        )

        # --- 위치 계산 ---
        current_centroid = centroids[c]
        vec_radial = current_centroid - global_center
        vec_repulsion = np.array([0.0, 0.0])
        crowding = 0

        for other_c, other_cent in centroids.items():
            if c == other_c:
                continue
            diff = current_centroid - other_cent
            dist = np.linalg.norm(diff)
            if dist > 1e-4:
                w = 1.0 / (dist**2.0)
                vec_repulsion += (diff / dist) * w
                crowding += w

        final_vec = vec_radial + (vec_repulsion * 4.0)
        if crowding > 0.5 or np.linalg.norm(vec_radial) < 1.0:
            final_vec[1] += (1.0 if i % 2 == 0 else -1.0) * 3.0

        norm = np.linalg.norm(final_vec)
        unit_vec = final_vec / norm if norm > 1e-4 else np.array([0, 1])

        hull_points = points[hull.vertices]
        anchor_point = hull_points[np.argmax(np.dot(hull_points, unit_vec))]

        offset = 2.5 + min(crowding, 3.5)
        text_pos = anchor_point + (unit_vec * offset)

        label_items.append(
            {
                "text_pos": text_pos,
                "anchor_point": anchor_point,
                # [변경] 라벨 텍스트에 Event Rate 표시
                "text_str": f"{risk_name}\n(Event:{event_rate:.2f})",
                "color": risk_color,
            }
        )

    # --- 물리 엔진 ---
    iterations = 50
    min_dist = 3.0
    for _ in range(iterations):
        for i in range(len(label_items)):
            for j in range(i + 1, len(label_items)):
                p1 = label_items[i]["text_pos"]
                p2 = label_items[j]["text_pos"]
                diff = p1 - p2
                dist = np.linalg.norm(diff)
                if dist < min_dist:
                    if dist < 1e-4:
                        force = np.array([0.1, 0.1])
                    else:
                        force = (diff / dist) * (min_dist - dist) * 0.5
                    label_items[i]["text_pos"] += force
                    label_items[j]["text_pos"] -= force

    # --- 축 확장 ---
    all_x = list(Z2[:, 0]) + [item["text_pos"][0] for item in label_items]
    all_y = list(Z2[:, 1]) + [item["text_pos"][1] for item in label_items]
    margin_x = (max(all_x) - min(all_x)) * 0.1
    margin_y = (max(all_y) - min(all_y)) * 0.1
    ax.set_xlim(min(all_x) - margin_x, max(all_x) + margin_x)
    ax.set_ylim(min(all_y) - margin_y, max(all_y) + margin_y)

    # # --- Annotation 그리기 ---
    # for item in label_items:
    #     bbox_props = dict(
    #         boxstyle="round,pad=0.2", fc="white", ec=item["color"], lw=1.5, alpha=0.95
    #     )

    #     ax.annotate(
    #         text=item["text_str"],
    #         xy=(item["anchor_point"][0], item["anchor_point"][1]),
    #         xytext=(item["text_pos"][0], item["text_pos"][1]),
    #         ha="center",
    #         va="center",
    #         fontsize=14,
    #         weight="bold",
    #         color=item["color"],
    #         bbox=bbox_props,
    #         arrowprops=dict(
    #             arrowstyle="-",
    #             color="black",
    #             linewidth=0.8,
    #             shrinkB=4,
    #             connectionstyle="arc3,rad=0.1",
    #         ),
    #         zorder=10,
    #     )

    # 범례 및 타이틀
    if -1 in uniq:
        legend_handles.insert(0, mpatches.Patch(color="#e0e0e0", label="Noise"))

    ax.legend(
        handles=legend_handles,
        title="Clusters",
        bbox_to_anchor=(1.02, 1),
        loc="upper left",
        borderaxespad=0,
        frameon=False,
        fontsize=14,
    )

    noise_pct = (labels == -1).sum() / len(labels) if len(labels) > 0 else 0
    full_title = f"{title}\n(Noise ratio={noise_pct:.1%})"

    ax.set_title(full_title, fontsize=14, weight="bold", pad=20)
    ax.set_xlabel("UMAP-1", fontsize=12, labelpad=10)
    ax.set_ylabel("UMAP-2", fontsize=12, labelpad=10)
    # ax.tick_params(axis='both', which='major', labelsize=12)

    ax.set_xticks([])
    ax.set_yticks([])

    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")


# =============== 유틸 ===============
def _standardize(X: np.ndarray) -> np.ndarray:
    X = np.asarray(X, dtype=float)
    X = np.nan_to_num(
        X, nan=0.0, posinf=np.finfo(float).max / 1e6, neginf=-np.finfo(float).max / 1e6
    )
    return (X - X.mean(0)) / (X.std(0) + 1e-9)


def save_png_svg(figpath: Path, **savefig_kwargs):
    plt.savefig(figpath, **savefig_kwargs)
    try:
        plt.savefig(
            figpath.with_suffix(".svg"),
            bbox_inches=savefig_kwargs.get("bbox_inches", "tight"),
        )
    except Exception:
        pass
    plt.close()


# =============== Youden 임계값 ===============
from sklearn.metrics import roc_curve


def _try_read_youden_threshold(label: str, base_dir: Path = OUT_ROOT.parent):
    for sep in ["__", "___"]:
        f = base_dir / f"{label}{sep}youden_metrics_Test.csv"
        if f.exists():
            try:
                dfy = pd.read_csv(f)
                if "threshold" in dfy.columns:
                    return float(dfy.loc[0, "threshold"])
            except Exception:
                pass
    return None


def _compute_youden_from_preds(y_true: np.ndarray, y_prob: np.ndarray) -> float:
    fpr, tpr, thr = roc_curve(y_true, y_prob)
    J = tpr - fpr
    return float(thr[int(np.argmax(J))])


# =============== SHAP 로딩 ===============
def get_shap_matrix(label: str):
    """
    Returns:
      expl: shap.Explanation (n, p)
      vals: ndarray (n, p)
      feat_names: list[str] length p (cleaned)
      y_prob: ndarray (n,)
      y_true: ndarray (n,)
      X_te_t: ndarray (n, p_after_preproc)
      df_pred: DataFrame with y_prob[, y_true, y_pred]
    """
    # ⚠️ 기존 프로젝트의 함수 가정
    expl, X_te_t, y_true, df_pred = shap_on_test(label)

    vals = np.asarray(expl.values, dtype=float)
    vals = np.nan_to_num(
        vals,
        nan=0.0,
        posinf=np.finfo(float).max / 1e6,
        neginf=-np.finfo(float).max / 1e6,
    )

    if getattr(expl, "feature_names", None) is not None:
        raw_names = list(expl.feature_names)
    else:
        raw_names = [f"f{i}" for i in range(vals.shape[1])]
    feat_names = [clean_name(n) for n in raw_names]

    y_prob = np.asarray(df_pred["y_prob"].values, dtype=float)
    y_true = np.asarray(getattr(y_true, "values", y_true)).astype(int)
    return expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred


# =============== k-distance & eps 추정 ===============
def _kdist_eps_heuristic(X_latent: np.ndarray, k: int = 5, pct: float = 0.85) -> float:
    nn = NearestNeighbors(n_neighbors=max(k, 2), metric="euclidean")
    nn.fit(X_latent)
    dists, _ = nn.kneighbors(X_latent)
    kth = np.sort(dists[:, -1])
    base_eps = float(np.quantile(kth, pct))
    return base_eps, kth


def save_kdist_plot(kth_sorted: np.ndarray, k: int, out_path: Path):
    plt.figure(figsize=(4, 3))
    plt.plot(np.arange(len(kth_sorted)), kth_sorted)
    plt.xlabel("Points (sorted)")
    plt.ylabel(f"{k}-NN distance")
    plt.title(f"k-distance curve (k={k})")
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")


# =============== 히트맵 ===============
def _agg_abs(values: np.ndarray, how="mean"):
    return (
        np.abs(values).mean(axis=0)
        if how == "mean"
        else np.mean(np.abs(values), axis=0)
    )


def plot_cluster_heatmap(
    vals,
    feat_names,
    labels,
    topN,
    out_path: Path,
    cmap: str = "rocket",
    agg: str = "mean",
):
    """
    반환:
      {
        "raw_all":  DataFrame (clusters × all_features) : 클러스터별 원시 집계 |SHAP|
        "raw_top":  DataFrame (clusters × topN)         : 히트맵에 사용된 TopN의 원시 |SHAP|
        "norm_top": DataFrame (clusters × topN)         : TopN의 행 정규화(상대값)
        "feat_top": List[str]                           : TopN 피처명
        "clusters": np.ndarray                          : 사용된 클러스터 ID (노이즈 -1 제외)
        "fig_raw_path": Path                            : raw heatmap PNG 경로
        "fig_norm_path": Path                           : normalized heatmap PNG 경로(= out_path)
      }
    """
    from textwrap import fill

    # 노이즈(-1) 제외
    clusters = [c for c in sorted(np.unique(labels)) if c != -1]
    if not clusters:
        return None

    # --- (A) 클러스터별 원시 집계 |SHAP|
    rows = []
    for c in clusters:
        mask = labels == c
        rows.append(_agg_abs(vals[mask], how=agg))  # shape: (p,)
    M = np.vstack(rows)  # (n_clusters, n_features)

    # --- (B) 히트맵에서 쓸 TopN 선택 (클러스터 max 기준)
    global_imp = _agg_abs(vals, how=agg)  # 전반적 중요도 (median|SHAP| 또는 mean|SHAP|)
    hetero = M.max(axis=0) - M.min(axis=0)  # 간단한 이질성 점수 (max - min)

    topK_imp = max(topN * 2, 30)  # 후보 풀(전반적 중요도 상위 K)
    cand = np.argsort(global_imp)[::-1][:topK_imp]
    top_idx = cand[
        np.argsort(hetero[cand])[::-1][:topN]
    ]  # 후보 안에서 이질성 큰 순 TopN

    M_top = M[:, top_idx]
    feat_top = [feat_names[i] for i in top_idx]

    ###########################################
    # global_imp = _agg_abs(vals, how=agg)  # shape (p,)  # median(|SHAP|)
    # top_idx = np.argsort(global_imp)[::-1][:topN]

    # M_top = M[:, top_idx]
    # feat_top = [feat_names[i] for i in top_idx]
    ###########################################
    # top_idx = np.argsort(M.max(axis=0))[::-1][:topN]
    # M_top = M[:, top_idx]  # raw topN
    # feat_top = [feat_names[i] for i in top_idx]

    # --- (C) 행 정규화(클러스터 내부 상대값)
    row_max = np.maximum(M_top.max(axis=1, keepdims=True), 1e-9)
    M_disp = M_top / row_max  # normalized topN

    # --- (D) DataFrame 구성
    idx_names = [f"cluster_{c}" for c in clusters]
    df_raw_all = pd.DataFrame(M, index=idx_names, columns=feat_names)
    df_raw_top = pd.DataFrame(M_top, index=idx_names, columns=feat_top)
    df_norm_top = pd.DataFrame(M_disp, index=idx_names, columns=feat_top)

    # ---------- (E) Normalized Heatmap (기존 파일명 유지) ----------
    xticks = [f"Cluster {c}" for c in clusters]
    yticks = [fill(f, width=28) for f in feat_top]
    fig_w = max(5, 0.8 * len(clusters))
    fig_h = max(4, 0.35 * len(yticks))

    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        df_norm_top.T.values,
        cmap=cmap,
        cbar_kws={"label": "Relative importance"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )
    ax.set_title(
        f"Cluster × Feature importance ({agg} |SHAP|)",  #  – Row-normalized
        fontsize=12,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features", fontsize=10, labelpad=6)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    ax.tick_params(axis="y", labelsize=8, pad=4)
    ax.collections[0].colorbar.ax.tick_params(labelsize=9)
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")

    # ---------- (F) Raw Heatmap (새 파일) ----------
    out_path_raw = out_path.with_name(out_path.stem + "__RAW.png")
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        df_raw_top.T.values,
        cmap=cmap,
        cbar_kws={"label": f"{agg} |SHAP|"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )
    ax.set_title(
        f"Cluster × Feature importance ({agg} |SHAP|)",
        fontsize=12,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features", fontsize=10, labelpad=6)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    ax.tick_params(axis="y", labelsize=8, pad=4)
    ax.collections[0].colorbar.ax.tick_params(labelsize=9)
    plt.tight_layout()
    save_png_svg(out_path_raw, dpi=300, bbox_inches="tight")

    return {
        "raw_all": df_raw_all,
        "raw_top": df_raw_top,
        "norm_top": df_norm_top,
        "feat_top": feat_top,
        "clusters": np.array(clusters),
        "fig_raw_path": out_path_raw,
        "fig_norm_path": out_path,
    }


# =============== 대표 케이스 ===============
def pick_representatives(y_prob: np.ndarray, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:  # 노이즈 제외(원하면 포함 가능)
            continue
        idx = np.where(labels == c)[0]
        probs = y_prob[idx]
        med = np.mean(probs)
        med_idx = idx[np.argmin(np.abs(probs - med))]
        max_idx = idx[np.argmax(probs)]
        min_idx = idx[np.argmin(probs)]
        reps.append(
            {
                "cluster": int(c),
                "mean_idx": int(med_idx),
                "max_idx": int(max_idx),
                "min_idx": int(min_idx),
            }
        )
    return reps


def pick_error_representatives(df_pred: pd.DataFrame, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:
            continue
        idx = np.where(labels == c)[0]
        d = df_pred.iloc[idx]
        rec = {"cluster": int(c)}
        fp = d[(d["y_true"] == 0) & (d["y_pred"] == 1)]
        fn = d[(d["y_true"] == 1) & (d["y_pred"] == 0)]
        if len(fp):
            rec["fp_idx"] = int(fp["y_prob"].idxmax())
        if len(fn):
            rec["fn_idx"] = int(fn["y_prob"].idxmin())
        reps.append(rec)
    return reps


def save_local_plots(
    label: str,
    expl: shap.Explanation,
    idx: int,
    out_dir: Path,
    feat_names=None,
    max_display=15,
    df_pred: pd.DataFrame | None = None,
    youden_thr: float | None = None,
    name_tag: str | None = None,
):
    out_dir.mkdir(parents=True, exist_ok=True)
    y_t = y_p = None
    y_prob = None
    thr_txt = ""
    if df_pred is not None:
        if "y_true" in df_pred.columns:
            y_t = int(df_pred.loc[idx, "y_true"])
        if "y_prob" in df_pred.columns:
            y_prob = float(df_pred.loc[idx, "y_prob"])
        if "y_pred" in df_pred.columns:
            y_p = int(df_pred.loc[idx, "y_pred"])
        else:
            thr = youden_thr if youden_thr is not None else 0.5
            y_p = (
                int((df_pred.loc[idx, "y_prob"]) >= thr)
                if "y_prob" in df_pred.columns
                else None
            )
        if youden_thr is not None:
            thr_txt = f", thr={youden_thr:.3f}"

    # waterfall
    plt.figure(figsize=(10, 8))
    try:
        shap.plots.waterfall(expl[idx], show=False, max_display=max_display)
    except Exception:
        row = expl[idx]
        shap.plots.waterfall(row, show=False, max_display=max_display)
    title_bits = [f"{label} | id={idx}"]
    if y_t is not None:
        title_bits.append(f"y_true={y_t}")
    if y_p is not None:
        title_bits.append(f"y_pred={y_p}")
    if y_prob is not None:
        title_bits.append(f"y_prob={y_prob:.3f}")
    if name_tag:
        title_bits.append(f"[{name_tag}]")
    ttl = " , ".join(title_bits) + thr_txt + " (Youden thr on TEST)"
    plt.title(ttl)
    plt.tight_layout()
    save_png_svg(
        out_dir
        / (
            f"{label}__waterfall_id{idx}"
            + (f"__{name_tag}" if name_tag else "")
            + ".png"
        ),
        dpi=300,
        bbox_inches="tight",
    )

    # force (옵션)
    try:
        plt.figure(figsize=(7, 2.8))
        shap.plots.force(
            expl[idx].base_values, expl[idx].values, matplotlib=True, show=False
        )
        plt.tight_layout()
        save_png_svg(
            out_dir
            / (
                f"{label}__force_id{idx}"
                + (f"__{name_tag}" if name_tag else "")
                + ".png"
            ),
            dpi=300,
        )
    except Exception:
        plt.close()


# =============== DBSCAN 통계/요약 ===============
def add_dbscan_stats(
    out_dir: Path,
    label: str,
    y_true: np.ndarray,
    y_prob: np.ndarray,
    labels: np.ndarray,
):
    lab = np.asarray(labels)
    clusters = [c for c in np.unique(lab) if c != -1]
    rows = []
    for c in clusters:
        m = lab == c
        rows.append(
            dict(
                cluster=int(c),
                n=int(m.sum()),
                event_rate=float(np.mean(y_true[m] == 1)),
                mean_y_prob=float(np.mean(y_prob[m])),
            )
        )
    df = pd.DataFrame(rows).sort_values("n", ascending=False)

    chi_p = np.nan
    kw_p = np.nan
    if len(clusters) >= 2:
        table = [
            [
                int(np.sum((lab == c) & (y_true == 1))),
                int(np.sum((lab == c) & (y_true == 0))),
            ]
            for c in clusters
        ]
        chi_p = chi2_contingency(table)[1]
        groups = [y_prob[lab == c] for c in clusters]
        kw_p = kruskal(*groups).pvalue

    df["p_event_rate(chi2)"] = chi_p
    df["p_yprob(kruskal)"] = kw_p
    df.to_csv(out_dir / f"{label}__dbscan_cluster_stats.csv", index=False)
    return df


# =============== 메인 루틴 ===============


def run_cluster_analysis_for_label(
    label: str,
    umap_n_components: int = 12,  # 군집용 UMAP 차원
    umap_n_neighbors: int = 12,
    umap_min_dist: float = 0.01,
    umap_metric: str = "euclidean",  # cosine
    min_samples: int = 6,  # DBSCAN
    eps: float | None = None,
    dbscan_metric: str = "euclidean",
    target_min_clusters: int = 2,
    max_noise_ratio: float = 0.45,
    max_trials: int = 5,
    topN: int = 10,
    agg_for_heatmap: str = "mean",
    save_representatives: bool = True,
    random_state: int = 42,
):
    assert HAS_UMAP, "umap-learn 패키지가 필요합니다."
    print(f"\n=== {label}: UMAP+DBSCAN (adaptive) ===")
    out_dir = OUT_ROOT / label
    out_dir.mkdir(parents=True, exist_ok=True)

    # 0) 데이터 로드
    expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred = get_shap_matrix(label)

    # Youden 임계값 & df_pred 보정
    ythr = _try_read_youden_threshold(label, base_dir=OUT_ROOT.parent)
    if ythr is None:
        ythr = _compute_youden_from_preds(
            np.asarray(y_true).astype(int), np.asarray(y_prob)
        )
    if "y_true" not in df_pred.columns:
        df_pred = df_pred.assign(y_true=np.asarray(y_true).astype(int))
    if "y_pred" not in df_pred.columns and "y_prob" in df_pred.columns:
        df_pred = df_pred.assign(y_pred=(df_pred["y_prob"] >= ythr).astype(int))

    # 1) 표준화 & UMAP(고차원; 군집용)
    # V = _standardize(vals) # 표준화
    V = vals
    um_hi = UMAP.UMAP(
        n_components=umap_n_components,
        n_neighbors=umap_n_neighbors,
        min_dist=umap_min_dist,
        metric=umap_metric,
        random_state=random_state,
    )
    X_hi = um_hi.fit_transform(V)

    # 2) eps 초기값 (k-distance) & 곡선 저장
    if eps is None:
        eps0, kth = _kdist_eps_heuristic(X_hi, k=min_samples, pct=0.9)  # 0.85
        eps = eps0
        save_kdist_plot(
            kth,
            k=min_samples,
            out_path=(out_dir / f"{label}__kdist_k{min_samples}.png"),
        )

    # 3) DBSCAN 적응형 루프
    trial = 0
    labels = None
    info_hist = []
    cur_min_samples = int(min_samples)
    cur_eps = float(eps)

    while trial < max_trials:
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)  # X_hi
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))

        info_hist.append(
            dict(
                trial=trial,
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

        if (n_clusters >= target_min_clusters) and (noise_ratio <= max_noise_ratio):
            break

        # 조정 규칙
        if noise_ratio > max_noise_ratio:
            cur_eps *= 1.20
            if cur_min_samples > 3:
                cur_min_samples -= 1
        elif n_clusters < target_min_clusters:
            cur_eps *= 0.88
        else:
            cur_eps *= 1.05

        trial += 1

    # all-noise 대비: 마지막 완화 시도 1회
    if (labels is None) or np.all(labels == -1):
        cur_eps *= 1.30
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)  # X_hi
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))
        info_hist.append(
            dict(
                trial="final_relax",
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

    # 4) 시각화용 2D UMAP (시각화 전용)
    if umap_n_components == 2:
        Z2 = X_hi
    else:
        um_2d = UMAP.UMAP(
            n_components=2,
            n_neighbors=max(umap_n_neighbors, 15),
            min_dist=max(umap_min_dist, 0.05),
            metric=umap_metric,
            random_state=random_state,
        )
        Z2 = um_2d.fit_transform(V)

    # [NEW] 위험도 순으로 라벨 재정렬 (이 부분 추가!)
    # Cluster 0 = Low Risk, Cluster N = High Risk
    if labels is not None:
        print("-> Reordering clusters by risk (0: Low -> N: High)...")
        labels = reorder_labels_by_risk(labels, np.asarray(y_true))

    # 5) 라벨 매핑 저장 (이제 정렬된 labels가 저장됨)
    m = pd.DataFrame({"idx": np.arange(len(labels)), "cluster": labels})
    m.to_csv(out_dir / f"{label}__dbscan_labels.csv", index=False)

    # # 5) 라벨 매핑 저장(노이즈 포함)
    # m = pd.DataFrame({"idx": np.arange(len(labels)), "cluster": labels})
    # m.to_csv(out_dir / f"{label}__dbscan_labels.csv", index=False)

    # # ⭐️⭐️⭐️⭐️기존 코드 6) 2D 플롯 (클러스터 색) ⭐️⭐️⭐️⭐️
    # uniq = sorted(np.unique(labels))
    # # 클러스터 개수가 많을 수 있으므로 tab10 사용 권장
    # cmap = plt.get_cmap("tab10")

    # # 범례 공간 확보를 위해 가로 사이즈를 약간 늘림
    # plt.figure(figsize=(8.5, 6.0))

    # # 1. 노이즈(-1) 먼저 그리기 (배경으로 깔리게 처리)
    # if -1 in uniq:
    #     mask = labels == -1
    #     plt.scatter(
    #         Z2[mask, 0],
    #         Z2[mask, 1],
    #         c="#e0e0e0",  # 아주 연한 회색
    #         edgecolor="#bbbbbb",  # 테두리는 조금 진하게
    #         linewidth=0.1,
    #         s=15,  # 크기는 작게
    #         alpha=0.4,  # 투명하게
    #         label="Noise",  # 범례 이름
    #         zorder=1,  # 가장 뒤쪽에 배치
    #     )

    # # 2. 실제 클러스터 그리기 Loop
    # for c in uniq:
    #     if c == -1:
    #         continue
    #     mask = labels == c
    #     color = cmap(c % 20)  # 색상 순환 적용

    #     plt.scatter(
    #         Z2[mask, 0],
    #         Z2[mask, 1],
    #         c=[color],  # 단일 색상 적용
    #         s=22,  # 클러스터 포인트는 조금 더 크게
    #         alpha=0.9,  # 진하게
    #         label=f"Cluster {c}",
    #         edgecolor="white",  # 포인트 구분감
    #         linewidth=0.3,
    #         zorder=2,  # 노이즈보다 앞쪽에 배치
    #     )

    # title = f"UMAP + DBSCAN ({umap_n_components}D)"
    # # 제목에 전체 대비 노이즈 비율 표기
    # n_noise = np.sum(labels == -1)
    # n_total = len(labels)
    # plt.title(
    #     f"{title}\nTotal: {n_total}, Clusters: {len(set(labels)-{-1})}, Noise: {n_noise} ({n_noise/n_total:.1%})",
    #     fontsize=11,
    # )
    # plt.xlabel("UMAP-1")
    # plt.ylabel("UMAP-2")

    # # 범례 설정 (그래프 영역 밖 우측 상단에 배치)
    # plt.legend(
    #     bbox_to_anchor=(1.02, 1),
    #     loc="upper left",
    #     borderaxespad=0,
    #     frameon=False,
    #     fontsize=9,
    #     markerscale=1.5,  # 범례의 점 크기 키우기
    # )

    # plt.tight_layout()
    # save_png_svg(
    #     out_dir / f"{label}__UMAP2D_DBSCAN_tuned.png", dpi=300, bbox_inches="tight"
    # )

    # 6) 2D 플롯 (Hull + Annotation 적용 버전)
    # print("-> Plotting UMAP with Threshold-based Risk Labels...")
    # plot_umap_with_hulls_and_labels(
    #     Z2=Z2,
    #     labels=labels,
    #     y_prob=y_prob,  # [중요] 여기에 y_prob를 반드시 넘겨주세요!
    #     out_path=out_dir / f"{label}__UMAP2D_DBSCAN_annotated.png",
    #     title=f"Risk Patterns ({label})",
    # )

    print("-> Plotting UMAP for PPT (Event Rate based)...")
    plot_umap_for_ppt(
        Z2=Z2,
        labels=labels,
        y_prob=y_prob,  # 기존 인자
        y_true=y_true,  # [NEW] 실제 정답값 전달
        out_path=out_dir / f"{label}__UMAP2D_DBSCAN_.png", # annotated_ppt
        title=f"Risk Patterns ({label})",
    )

    # 6-2) 2D 플롯 (위험도 색)
    plt.figure(figsize=(7.2, 6.2))
    sc = plt.scatter(Z2[:, 0], Z2[:, 1], c=y_prob, s=18, alpha=0.9, cmap="plasma")
    plt.colorbar(sc, label="Predicted risk (prob.)")
    plt.title(f"UMAP(2D) (colored by risk) — {label}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.margins(0.02)
    save_png_svg(
        out_dir / f"{label}__UMAP2D_risk_DBSCAN.png", dpi=300, bbox_inches="tight"
    )

    # 7) 클러스터 통계/요약 + 히트맵
    df_stats = add_dbscan_stats(
        out_dir,
        label,
        y_true=np.asarray(y_true),
        y_prob=np.asarray(y_prob),
        labels=np.asarray(labels),
    )
    ret = plot_cluster_heatmap(
        vals,
        feat_names,
        labels,
        topN=topN,
        out_path=out_dir / f"{label}__heatmap_top{topN}_dbscan.png",
        cmap="magma_r",  # rocket_r, viridis_r
        agg=agg_for_heatmap,
    )

    # === 수치 저장(요청 파일명 규칙) ===
    if ret is not None:
        # 모든 피처의 원시 |SHAP| (클러스터 × 전체 피처)
        ret["raw_all"].to_csv(
            out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__ALL_FEATURES.csv",
            index=True,
        )
        # 히트맵에 쓰인 TopN의 원시 |SHAP|
        ret["raw_top"].to_csv(
            out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_RAW.csv",
            index=True,
        )
        # 히트맵에 쓰인 TopN의 행 정규화(상대값)
        ret["norm_top"].to_csv(
            out_dir
            / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_ROWNORM.csv",
            index=True,
        )

    # 8) 대표 케이스 저장 (mean/max/min + FP/FN)
    if save_representatives:
        rep_dir = out_dir / "representatives_dbscan"
        # mean/max/min
        reps = pick_representatives(y_prob, np.asarray(labels))
        for rep in reps:
            c = rep["cluster"]
            for tag in ["mean_idx", "max_idx", "min_idx"]:
                i = rep[tag]
                name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                save_local_plots(
                    label,
                    expl,
                    i,
                    rep_dir / f"cluster_{c}",
                    feat_names=feat_names,
                    max_display=15,
                    df_pred=df_pred,
                    youden_thr=ythr,
                    name_tag=name_tag,
                )
        # FP/FN
        err_reps = pick_error_representatives(df_pred, np.asarray(labels))
        for rep in err_reps:
            c = rep["cluster"]
            for tag in ["fp_idx", "fn_idx"]:
                if tag in rep:
                    i = rep[tag]
                    name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                    save_local_plots(
                        label,
                        expl,
                        i,
                        rep_dir / f"cluster_{c}",
                        feat_names=feat_names,
                        max_display=15,
                        df_pred=df_pred,
                        youden_thr=ythr,
                        name_tag=name_tag,
                    )

    # 9) 메타(JSON)
    meta = dict(
        timestamp=time.strftime("%Y-%m-%d %H:%M:%S"),
        algo="dbscan_tuned",
        random_state=random_state,
        umap=dict(
            n_components=umap_n_components,
            n_neighbors=umap_n_neighbors,
            min_dist=umap_min_dist,
            metric=umap_metric,
        ),
        final=dict(eps=float(cur_eps), min_samples=int(cur_min_samples)),
        trials=len([h for h in info_hist if isinstance(h.get("trial"), int)]),
        n_clusters=int(len(set(labels) - {-1})),
        noise_ratio=float(np.mean(np.asarray(labels) == -1)),
        history=info_hist,
    )
    with open(out_dir / f"{label}__dbscan_meta.json", "w") as f:
        json.dump(meta, f, indent=2)

    print(f"-> saved: {out_dir}")
    return df_stats, labels

In [ ]:
# =============== 전체 라벨 반복 실행 ===============
all_cluster_summaries = []
for lb in LABELS:
    try:
        df_sum, labels = run_cluster_analysis_for_label(
            lb,
            umap_n_components=2,
            umap_n_neighbors=45,  # 40
            umap_min_dist=0.0,  # 0.05, 0.01
            umap_metric="euclidean",  # cosine
            min_samples=17,  # 20
            eps=None,
            dbscan_metric="euclidean",
            target_min_clusters=3,
            max_noise_ratio=0.45,  # 0.45
            max_trials=5,
            topN=10,
            agg_for_heatmap="mean",
            save_representatives=True,
            random_state=42,
        )
        df_sum.insert(0, "label", lb)
        all_cluster_summaries.append(df_sum)
    except Exception as e:
        print(f"[WARN] {lb}: {e}")

if all_cluster_summaries:
    all_summ = pd.concat(all_cluster_summaries, ignore_index=True)
    out_csv = OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.csv"
    all_summ.to_csv(out_csv, index=False)
    print("✅ saved:", out_csv)
    # Excel(라벨별 시트) 옵션
    try:
        with pd.ExcelWriter(
            OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.xlsx"
        ) as xw:
            for lb, df_ in all_summ.groupby("label"):
                df_.to_excel(xw, sheet_name=lb, index=False)
        print("✅ saved:", OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.xlsx")
    except Exception as e:
        print("[WARN] Excel save:", e)

In [ ]:
# ---------- 경로 설정 ----------
csv_path = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters/ALL_labels__cluster_summary_dbscan_tuned.csv")
)
out_path = csv_path.with_name("Table3_Cluster_Summary.xlsx")

# ---------- 데이터 로드 ----------
df = pd.read_csv(csv_path)

# ---------- 전처리 ----------
# (1) Δ 계산: 예측확률 - 실제 event rate 차이
df["Delta(abs)"] = (df["mean_y_prob"] - df["event_rate"]).abs().round(3)
df["Direction"] = df.apply(
    lambda x: (
        "Over (Pred>Event)"
        if x["mean_y_prob"] > x["event_rate"]
        else "Under (Pred<Event)"
    ),
    axis=1,
)

# (2) 도메인/해석 컬럼 자리 만들기 (수동 입력용)
df["Dominant domain"] = ""
df["Key top features"] = ""
df["Interpretation summary"] = ""

# (3) 정렬 및 보기 좋은 라벨링
df = df.rename(
    columns={
        "label": "Outcome (label)",
        "cluster": "Cluster",
        "n": "N",
        "event_rate": "Event rate",
        "mean_y_prob": "Mean predicted prob",
        "p_event_rate(chi2)": "p (event rate, χ²)",
        "p_yprob(kruskal)": "p (pred prob, KW)",
    }
)
df = df[
    [
        "Outcome (label)",
        "Cluster",
        "N",
        "Event rate",
        "Mean predicted prob",
        "Delta(abs)",
        "Direction",
        "Dominant domain",
        "Key top features",
        "Interpretation summary",
        "p (event rate, χ²)",
        "p (pred prob, KW)",
    ]
]

# ---------- Excel 저장 ----------
with pd.ExcelWriter(out_path) as writer:
    for lb, dsub in df.groupby("Outcome (label)"):
        dsub.to_excel(writer, sheet_name=lb, index=False)
    # 전체 통합 시트
    df.to_excel(writer, sheet_name="All_labels_combined", index=False)

print(f"✅ Saved Table 3 summary Excel → {out_path}")

In [ ]:
# ===== 경로 설정 =====
base = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters")
)


# ===== Domain 분류 함수 =====
def infer_domain(feat: str) -> str:
    t = feat.lower()
    if any(k in t for k in ["aggression", "support", "social", "interpersonal", "llm"]):
        return "SDoMH"
    if any(k in t for k in ["er visit", "≥2 er", "≥3 er", "admission", "admit"]):
        return "Utilization"
    if any(
        k in t
        for k in [
            "bl3",
            "serum",
            "cholesterol",
            "hdl",
            "ldl",
            "triglyceride",
            "crp",
            "anc",
            "sodium",
            "na",
            "potassium",
            "k(",
            "hematocrit",
            "mchc",
            "eosinophil",
            "basophil",
            "alc",
            "lymphocyte",
        ]
    ):
        return "Lab"
    if any(
        k in t
        for k in [
            "mmpi",
            "hy",
            "mf",
            "vrin",
            "trinf",
            "psychometric",
            "audit",
            "hcl",
            "asrs",
            "ham",
        ]
    ):
        return "Psychometric"
    if any(
        k in t
        for k in [
            "olanz",
            "lith",
            "bzd",
            "quetiap",
            "antipsych",
            "treatment",
            "medic",
            "benzodiazepine",
        ]
    ):
        return "Treatment"
    if "sleep" in t:
        return "Sleep"
    if any(
        k in t
        for k in [
            "sex",
            "marital",
            "age",
            "education",
            "job",
            "drinking",
            "psychiatric family history",
            "hospitalization period",
        ]
    ):
        return "Demographic/Psychosocial"
    return "Other"


# ===== 초기화 =====
labels = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]
records_summary = []  # Table 4
etable_dict = {}  # eTable S4 outcome별 sheet 저장

# ===== outcome별 반복 =====
for lb in labels:
    fpath = base / lb / f"{lb}__heatmap_values_mean__TOP10_RAW.csv"
    print("using:", fpath)
    df = pd.read_csv(fpath, index_col=0)

    # --- eTable S4용 wide-format 저장
    df_wide = df.copy()
    df_wide["Feature"] = df_wide.index
    etable_dict[lb] = df_wide.T  # transpose해서 Feature가 column으로 가도록 저장

    # --- Table 4용 계산
    for col in df.columns:
        vals = df[col].values
        # Kruskal–Wallis 검정 (클러스터 간 이질성 유의성)
        try:
            stat, pval = kruskal(*[df[col][df.index == c] for c in df.index])
        except Exception:
            pval = None

        records_summary.append(
            {
                "Outcome": lb,
                "Feature": col,
                "Mean(|SHAP|)": vals.mean(),
                "SD": vals.std(),
                "Range": vals.max() - vals.min(),
                "Δ(Top−Bottom)": vals.max() - vals.min(),
                "Max cluster": df.index[vals.argmax()],
                "Min cluster": df.index[vals.argmin()],
                "Top vs Bottom": f"{df.index[vals.argmax()]} > {df.index[vals.argmin()]}",
                "p(KW)": pval,
            }
        )

# ===== Table 4 (요약) =====
df_table4 = pd.DataFrame(records_summary)
df_table4["Domain"] = df_table4["Feature"].apply(infer_domain)
df_table4 = df_table4.sort_values(["Outcome", "SD"], ascending=[True, False])

# ===== 저장 =====
out_summary = base / "Table4_CrossCluster_SHAP_Heterogeneity.xlsx"
out_wide = base / "eTableS4_SHAP_ClusterMatrix.xlsx"

with pd.ExcelWriter(out_summary) as writer:
    df_table4.to_excel(writer, sheet_name="All_outcomes", index=False)
    for lb, sub in df_table4.groupby("Outcome"):
        sub.to_excel(writer, sheet_name=lb, index=False)

with pd.ExcelWriter(out_wide) as writer:
    for lb, dfw in etable_dict.items():
        dfw.to_excel(writer, sheet_name=lb)

print(f"✅ Saved Table 4 summary → {out_summary}")
print(f"✅ Saved eTable S4 matrix → {out_wide}")

In [ ]:
# ===== 경로 =====
base = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters")
)

# Table4 요약 엑셀
f_table4 = base / "Table4_CrossCluster_SHAP_Heterogeneity.xlsx"
df = pd.read_excel(f_table4, sheet_name="All_outcomes")

sns.set(style="whitegrid", font_scale=1.0)
palette = sns.color_palette("Set2")

# ===== (A) Feature별 Boxplot (x=cluster, y=mean |SHAP|) =====
for outcome in ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]:
    target_outcome = outcome
    fpath = (
        base / target_outcome / f"{target_outcome}__heatmap_values_mean__TOP10_RAW.csv"
    )
    df_feat = pd.read_csv(fpath, index_col=0)
    df_feat.index = [
        c.replace("cluster_", "C") for c in df_feat.index
    ]  # cluster_0 → C0

    # ===== (1) Dotplot (by Feature 색상) =====
    df_melt = df_feat.T.reset_index().melt(
        id_vars="index", var_name="Cluster", value_name="Mean |SHAP|"
    )

    plt.figure(figsize=(8, 5))
    ax = sns.stripplot(
        x="Cluster",
        y="Mean |SHAP|",
        hue="index",
        data=df_melt,
        dodge=True,
        size=6,
        jitter=False,
        palette="tab10",
        alpha=0.9,
    )

    # ➊ 카테고리 순서(좌→우) 얻기
    cats = [t.get_text() for t in ax.get_xticklabels()]

    # ➋ 각 카테고리 경계(정수 + 0.5 위치)에 점선 추가
    for i in range(len(cats) - 1):
        ax.axvline(i + 0.5, ls="--", lw=0.8, color="gray", alpha=0.5, zorder=0)

    # ➌ 레이아웃/레이블
    ax.set_title(
        f"{target_outcome} — Cluster mean |SHAP| per Feature",
        fontsize=11,
        weight="bold",
    )
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Mean |SHAP|")
    plt.xticks()  # rotation=30
    plt.legend(title="Feature", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.savefig(base / f"Fig5A_{target_outcome}_dotplot_byFeature_guides.png", dpi=300)

    # ===== (2) Top N feature subplot 버전 =====
    top_feats = df_feat.columns[:6]  # 상위 6개 feature 예시
    fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharey=True)

    for ax, feat in zip(axes.flat, top_feats):
        sns.lineplot(x=df_feat.index, y=df_feat[feat], marker="o", color="black", ax=ax)
        ax.set_title(feat, fontsize=10)
        ax.set_xlabel("Cluster")
        ax.set_ylabel("Mean |SHAP|")
        ax.set_xticklabels(ax.get_xticklabels(), ha="right")  # rotation=30

    plt.suptitle(
        f"{target_outcome} — Cluster mean |SHAP| (Top 6 features)",
        fontsize=13,
        weight="bold",
    )
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.savefig(base / f"Fig5A_{target_outcome}_TopFeatures_clean.png", dpi=300)

# ===== (B) Δ(Top−Bottom) Barplot (feature별 이질성 순위) =====
topN = 20  # 상위 20개 표시 (필요시 조절)
df_rank = df.sort_values("Δ(Top−Bottom)", ascending=False).head(topN)

plt.figure(figsize=(7.5, 6.5))
sns.barplot(
    y="Feature",
    x="Δ(Top−Bottom)",
    hue="Domain",
    data=df_rank,
    palette=palette,
    dodge=False,
)
plt.title(
    "Top features by SHAP heterogeneity (Δ Top−Bottom)", fontsize=12, weight="bold"
)
plt.xlabel("Δ (max−min mean |SHAP| across clusters)")
plt.ylabel("Feature")
plt.legend(title="Domain", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.savefig(base / "Fig5B_SHAP_Heterogeneity_Rank.png", dpi=300)

# ===== (C) Domain별 평균 SD 막대그래프 =====
df_dom = (
    df.groupby("Domain", as_index=False)["SD"].mean().sort_values("SD", ascending=False)
)

plt.figure(figsize=(6.5, 4.2))
sns.barplot(y="Domain", x="SD", data=df_dom, palette=palette)
plt.title("Average SHAP heterogeneity (SD) by Domain", fontsize=12, weight="bold")
plt.xlabel("Mean SD of mean |SHAP| across clusters")
plt.ylabel("")
plt.tight_layout()
plt.savefig(base / "Fig5C_Domain_SHAP_Heterogeneity.png", dpi=300)

print("✅ Figures saved:")
print(" -", base / f"Fig5A_{target_outcome}_boxplot.png")
print(" -", base / "Fig5B_SHAP_Heterogeneity_Rank.png")

print(" -", base / "Fig5C_Domain_SHAP_Heterogeneity.png")

# Raw SHAP 뽑기

In [ ]:
# === 샘플별 SHAP 추출 & 저장 ===
OUT_ROOT = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters/SHAP")
)


def export_sample_shap(label: str, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)

    # 1) SHAP/예측 로드 (이미 있는 함수들 활용)
    expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred = get_shap_matrix(label)

    # 2) 메타(정답/확률/예측) 정리
    meta = pd.DataFrame(
        {
            "sample_id": np.arange(vals.shape[0], dtype=int),
            "y_true": np.asarray(y_true, dtype=int),
            "y_prob": np.asarray(y_prob, dtype=float),
        }
    )
    if "y_pred" in df_pred.columns:
        meta["y_pred"] = df_pred["y_pred"].astype(int)
    else:
        # 필요 시 Youden threshold로 예측치 생성
        thr = _try_read_youden_threshold(label, base_dir=OUT_ROOT.parent)
        if thr is None:
            thr = _compute_youden_from_preds(
                meta["y_true"].values, meta["y_prob"].values
            )
        meta["y_pred"] = (meta["y_prob"] >= thr).astype(int)

    # 3) 샘플×피처 SHAP (wide)
    df_shap_wide = pd.DataFrame(vals, columns=feat_names)
    df_wide = pd.concat([meta, df_shap_wide], axis=1)

    # 4) (선택) 클러스터 라벨 붙이기: 이미 DBSCAN 돌렸다면 라벨 파일을 자동 병합
    lab_path = OUT_ROOT / label / f"{label}__dbscan_labels.csv"
    if lab_path.exists():
        lab = pd.read_csv(lab_path)  # columns: idx, cluster
        lab = lab.rename(columns={"idx": "sample_id"})
        df_wide = df_wide.merge(lab, on="sample_id", how="left")

    # 5) 저장 (wide & long)
    out_wide_csv = OUT_ROOT / label / f"{label}__sample_SHAP_wide.csv"
    out_wide_parq = OUT_ROOT / label / f"{label}__sample_SHAP_wide.parquet"
    df_wide.to_csv(out_wide_csv, index=False)
    try:
        df_wide.to_parquet(out_wide_parq, index=False)
    except Exception:
        pass

    # long 포맷 (원하면 주석 해제; 용량 커질 수 있음)
    # df_long = df_wide.melt(
    #     id_vars=[c for c in ["sample_id","y_true","y_prob","y_pred","cluster"] if c in df_wide.columns],
    #     var_name="feature",
    #     value_name="shap_value"
    # )
    # df_long.to_parquet(OUT_ROOT / label / f"{label}__sample_SHAP_long.parquet", index=False)

    print(f"✅ saved: {out_wide_csv}")
    return df_wide


for label in ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]:
    export_sample_shap(label, OUT_ROOT / label)

# Cluster 유의성 비교

In [ ]:
from pathlib import Path
from itertools import combinations
import numpy as np
import pandas as pd
from scipy.stats import kruskal, mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [ ]:
# --------------------------
# 경로
# --------------------------
ROOT = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters")
)
SHAP_ROOT = ROOT / "SHAP"
DBSCAN_ROOT = ROOT
SIG_ROOT = DBSCAN_ROOT / "cluster_sig"
SIG_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# --------------------------
# 유틸
# --------------------------
META_COLS = {"sample_id", "y_true", "y_prob", "y_pred", "cluster"}


def _feature_columns(df: pd.DataFrame):
    return [c for c in df.columns if c not in META_COLS]


def _all_identical(groups):
    cat = np.concatenate([g.ravel() for g in groups if len(g) > 0])
    return cat.size == 0 or np.nanstd(cat) == 0.0


def _safe_mw(a, b):
    if len(a) < 2 or len(b) < 2:
        return np.nan, np.nan
    if _all_identical([a, b]):
        return 1.0, 0.0
    p = mannwhitneyu(a, b, alternative="two-sided").pvalue
    # Cliff's delta
    gt = sum(x > y for x in a for y in b)
    lt = sum(x < y for x in a for y in b)
    d = (gt - lt) / (len(a) * len(b))
    return p, d


# --------------------------
# 1) SHAP × Cluster 병합
# --------------------------
def attach_cluster_fast(label: str) -> Path:
    """
    sample_id == idx 가정 하에 DBSCAN 라벨을 병합.
    병합된 CSV를 SHAP_ROOT/<label>/에 저장하고 그 경로를 반환.
    """
    shap_dir = SHAP_ROOT / label
    dbscan_dir = DBSCAN_ROOT / label
    shap_dir.mkdir(parents=True, exist_ok=True)

    f_shap = shap_dir / f"{label}__sample_SHAP_wide.csv"
    f_lab = dbscan_dir / f"{label}__dbscan_labels.csv"
    assert f_shap.exists(), f"Not found: {f_shap}"
    assert f_lab.exists(), f"Not found: {f_lab}"

    df = pd.read_csv(f_shap)
    lab = pd.read_csv(f_lab)  # columns: idx, cluster

    merged = df.merge(
        lab.rename(columns={"idx": "sample_id"}), on="sample_id", how="left"
    )
    cov = merged["cluster"].notna().mean()
    print(f"[{label}] coverage by sample_id==idx: {cov*100:.1f}%")

    out = shap_dir / f"{label}__sample_SHAP_wide_with_cluster.csv"
    merged.to_csv(out, index=False)
    print(f"✅ saved (merged): {out}")
    return out


# --------------------------
# 2) 유의성 검정 (Kruskal + 모든 pairwise)
#    - noise(-1) 제외 기본값
#    - pairwise FDR은 '피처별'로 보정
# --------------------------
def run_significance_full(
    label: str, use_abs: bool = True, include_noise: bool = False
) -> Path:
    f_merged = SHAP_ROOT / label / f"{label}__sample_SHAP_wide_with_cluster.csv"
    df = pd.read_csv(f_merged)
    if not include_noise:
        df = df[df["cluster"] != -1].copy()
    print(f"[{label}] Used {len(df)} samples (exclude noise={not include_noise})")

    feat_cols = _feature_columns(df)
    clusters = sorted(df["cluster"].dropna().unique())

    # ---- (A) Kruskal ----
    rows_kw = []
    for feat in feat_cols:
        vals = np.abs(df[feat].values) if use_abs else df[feat].values
        groups = [vals[df["cluster"].values == c] for c in clusters]
        if len(groups) < 2 or any(len(g) < 2 for g in groups) or _all_identical(groups):
            p_kw = np.nan
        else:
            try:
                p_kw = kruskal(*groups).pvalue
            except Exception:
                p_kw = np.nan
        rows_kw.append({"Feature": feat, "p_kw": p_kw})
    kw = pd.DataFrame(rows_kw)
    kw["q_kw"] = multipletests(kw["p_kw"].fillna(1.0), method="fdr_bh")[1]

    # 보조 요약(중앙값 기반 Top/Bottom 및 범위)
    if len(clusters) >= 1:
        med = df.groupby("cluster")[feat_cols].mean()
        rng = med.max(0) - med.min(0)
        top_c = med.idxmax()
        bot_c = med.idxmin()
        kw["Δ(Top−Bottom)"] = kw["Feature"].map(rng.to_dict())
        kw["Top cluster"] = kw["Feature"].map(top_c.to_dict())
        kw["Bottom cluster"] = kw["Feature"].map(bot_c.to_dict())
        kw["Top vs Bottom"] = (
            kw["Top cluster"].astype(str) + " > " + kw["Bottom cluster"].astype(str)
        )

    # ---- (B) Pairwise 모든 클러스터 쌍 (피처별 FDR) ----
    pair_tables = []
    for feat in feat_cols:
        vals = np.abs(df[feat].values) if use_abs else df[feat].values
        rows = []
        for c1, c2 in combinations(clusters, 2):
            a, b = vals[df["cluster"] == c1], vals[df["cluster"] == c2]
            p, d = _safe_mw(a, b)
            rows.append(
                {
                    "Feature": feat,
                    "Cluster1": c1,
                    "Cluster2": c2,
                    "p_raw": p,
                    "cliffs_delta": d,
                    "n1": len(a),
                    "n2": len(b),
                }
            )
        pw_feat = pd.DataFrame(rows)
        if len(pw_feat) > 0:
            pw_feat["q_fdr"] = multipletests(
                pw_feat["p_raw"].fillna(1.0), method="fdr_bh"
            )[1]
        pair_tables.append(pw_feat)
    pw = (
        pd.concat(pair_tables, ignore_index=True)
        if pair_tables
        else pd.DataFrame(
            columns=[
                "Feature",
                "Cluster1",
                "Cluster2",
                "p_raw",
                "cliffs_delta",
                "n1",
                "n2",
                "q_fdr",
            ]
        )
    )

    # ---- (C) 저장 ----
    out_xlsx = SIG_ROOT / f"{label}__SHAP_cluster_significance_full.xlsx"
    with pd.ExcelWriter(out_xlsx) as xw:
        kw.to_excel(xw, sheet_name="Kruskal", index=False)
        pw.to_excel(xw, sheet_name="Pairwise_All", index=False)
    print(f"✅ saved full pairwise significance: {out_xlsx}")
    return out_xlsx


# --------------------------
# 3) 전체 outcome 일괄 실행
# --------------------------
LABELS = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]


def run_all(labels=LABELS):
    for lb in labels:
        try:
            attach_cluster_fast(lb)
            run_significance_full(
                lb, use_abs=True, include_noise=False
            )  # |SHAP| 기준, 노이즈 제외
        except Exception as e:
            print(f"[WARN] {lb}: {e}")

In [ ]:
# 실행
if __name__ == "__main__":
    run_all()

## Table 및 시각화 반영

In [ ]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import kruskal  # 기존 코드에서 사용

sns.set(style="whitegrid", font_scale=1.0)

In [ ]:
# ----- 경로 설정 -----
RAWNORM_BASE = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters")
)
SIG_ROOT = RAWNORM_BASE / "cluster_sig"
FIGTAB_ROOT = RAWNORM_BASE / "cluster_figure_table"
FIGTAB_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# ----- 유틸 -----
def stars_from_q(q):
    if pd.isna(q):
        return ""
    if q < 0.001:
        return "***"
    if q < 0.01:
        return "**"
    if q < 0.05:
        return "*"
    return ""


def parse_cluster_str(s):
    # "cluster_3" -> 3 / "C3" -> 3
    if pd.isna(s):
        return None
    m = re.search(r"(-?\d+)", str(s))
    return int(m.group(1)) if m else None


# ===== Domain 분류 함수 (기존과 동일) =====
# ===== Domain 분류 함수 =====
def infer_domain(feat: str) -> str:
    t = feat.lower()
    if any(k in t for k in ["aggression", "support", "social", "interpersonal", "llm"]):
        return "SDoMH"
    if any(k in t for k in ["er visit", "≥2 er", "≥3 er", "admission", "admit"]):
        return "Utilization"
    if any(
        k in t
        for k in [
            "bl3",
            "serum",
            "cholesterol",
            "hdl",
            "ldl",
            "triglyceride",
            "crp",
            "anc",
            "sodium",
            "na",
            "potassium",
            "k(",
            "hematocrit",
            "mchc",
            "eosinophil",
            "basophil",
            "alc",
            "lymphocyte",
        ]
    ):
        return "Lab"
    if any(
        k in t
        for k in [
            "mmpi",
            "hy",
            "mf",
            "vrin",
            "trinf",
            "psychometric",
            "audit",
            "hcl",
            "asrs",
            "ham",
        ]
    ):
        return "Psychometric"
    if any(
        k in t
        for k in [
            "olanz",
            "lith",
            "bzd",
            "quetiap",
            "antipsych",
            "treatment",
            "medic",
            "benzodiazepine",
        ]
    ):
        return "Treatment"
    if "sleep" in t:
        return "Sleep"
    if any(
        k in t
        for k in [
            "sex",
            "marital",
            "age",
            "education",
            "job",
            "drinking",
            "psychiatric family history",
            "hospitalization period",
        ]
    ):
        return "Demographic/Psychosocial"
    return "Other"


# ===== 1) 기존 Table4 요약 읽기 =====
f_table4 = RAWNORM_BASE / "Table4_CrossCluster_SHAP_Heterogeneity.xlsx"
df_all = pd.read_excel(f_table4, sheet_name="All_outcomes")
# 안전하게 Domain 재계산(혹시 비어있을 경우)
if "Domain" not in df_all.columns:
    df_all["Domain"] = df_all["Feature"].apply(infer_domain)

# ===== 2) 유의성 결과(각 label)와 결합 → 메인 테이블 확장 =====
LABELS = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]

# 결과 컬럼 초기화
for col in ["p_kw", "q_kw", "p_tb", "q_tb", "cliffs_delta", "sig_kw", "sig_tb"]:
    if col not in df_all.columns:
        df_all[col] = np.nan

# label별로 KW + Pairwise에서 Top-Bottom p를 주입
for lb in LABELS:
    sig_path = SIG_ROOT / f"{lb}__SHAP_cluster_significance_full.xlsx"
    if not sig_path.exists():
        print(f"[WARN] missing significance file: {sig_path}")
        continue
    df_kw = pd.read_excel(sig_path, sheet_name="Kruskal")
    df_pw = pd.read_excel(sig_path, sheet_name="Pairwise_All")

    # 매핑(dict) 준비
    qkw_map = dict(zip(df_kw["Feature"], df_kw["q_kw"]))
    pkw_map = dict(zip(df_kw["Feature"], df_kw["p_kw"]))

    # df_all 중 해당 outcome만 추림
    m_out = df_all["Outcome"] == lb
    sub = df_all.loc[m_out].copy()

    # 각 row에서 Top/Bottom 클러스터 번호 파싱 후 pairwise에서 해당 쌍의 p/q/효과크기 찾기
    p_tb_list, q_tb_list, d_tb_list, sig_tb_list = [], [], [], []
    p_kw_list, q_kw_list, sig_kw_list = [], [], []
    for _, row in sub.iterrows():
        feat = row["Feature"]
        # KW
        pkw = pkw_map.get(feat, np.nan)
        qkw = qkw_map.get(feat, np.nan)
        sig_kw = stars_from_q(qkw)

        # Top/Bottom 클러스터
        topc = parse_cluster_str(row.get("Max cluster") or row.get("Top cluster") or "")
        botc = parse_cluster_str(
            row.get("Min cluster") or row.get("Bottom cluster") or ""
        )

        # pairwise에서 두 방향(Cluster1,Cluster2) 모두 확인
        df_feat = df_pw[df_pw["Feature"] == feat]
        hit = df_feat[
            ((df_feat["Cluster1"] == topc) & (df_feat["Cluster2"] == botc))
            | ((df_feat["Cluster1"] == botc) & (df_feat["Cluster2"] == topc))
        ]
        if len(hit):
            p_tb = float(hit["p_raw"].iloc[0])
            q_tb = float(hit["q_fdr"].iloc[0])
            d_tb = float(hit["cliffs_delta"].iloc[0])
            sig_tb = stars_from_q(q_tb)
        else:
            p_tb = q_tb = d_tb = np.nan
            sig_tb = ""

        p_kw_list.append(pkw)
        q_kw_list.append(qkw)
        sig_kw_list.append(sig_kw)
        p_tb_list.append(p_tb)
        q_tb_list.append(q_tb)
        d_tb_list.append(d_tb)
        sig_tb_list.append(sig_tb)

    # 주입
    df_all.loc[m_out, "p_kw"] = p_kw_list
    df_all.loc[m_out, "q_kw"] = q_kw_list
    df_all.loc[m_out, "sig_kw"] = sig_kw_list
    df_all.loc[m_out, "p_tb"] = p_tb_list
    df_all.loc[m_out, "q_tb"] = q_tb_list
    df_all.loc[m_out, "cliffs_delta"] = d_tb_list
    df_all.loc[m_out, "sig_tb"] = sig_tb_list

# 저장(메인 테이블)
out_table_main = FIGTAB_ROOT / "Table4_with_significance.xlsx"
with pd.ExcelWriter(out_table_main) as xw:
    df_all.to_excel(xw, sheet_name="All_outcomes", index=False)
    for lb, sub in df_all.groupby("Outcome"):
        sub.to_excel(xw, sheet_name=lb, index=False)
print("✅ Saved main table with significance →", out_table_main)

# ===== 3) 서플: 모든 쌍 pairwise를 그대로 outcome별로 CSV 저장 =====
for lb in LABELS:
    sig_path = SIG_ROOT / f"{lb}__SHAP_cluster_significance_full.xlsx"
    if sig_path.exists():
        df_pw = pd.read_excel(sig_path, sheet_name="Pairwise_All")
        df_pw.to_csv(FIGTAB_ROOT / f"Supp_Pairwise_All_{lb}.csv", index=False)

# ===== 4) 그림에 별표 추가 (KW 기준) =====
palette = sns.color_palette("Set2")

# (A) Feature별 Dotplot: 범례(Feature 이름)에 별표 붙이기 (q_kw 기준)
for outcome in LABELS:
    # 히트맵 수치 파일 로드
    fpath = RAWNORM_BASE / outcome / f"{outcome}__heatmap_values_mean__TOP10_RAW.csv"
    if not fpath.exists():
        print(f"[WARN] missing heatmap CSV: {fpath}")
        continue
    df_feat = pd.read_csv(fpath, index_col=0)
    # C 라벨로 보기 좋게
    df_feat.index = [c.replace("cluster_", "C") for c in df_feat.index]

    # 해당 outcome의 q_kw 맵(Feature→별표)
    sub_main = df_all[df_all["Outcome"] == outcome]
    qkw_map = dict(zip(sub_main["Feature"], sub_main["q_kw"]))
    star_map = {f: stars_from_q(q) for f, q in qkw_map.items()}

    # legend용 라벨 교체를 위해 컬럼명을 "원래+별"로 임시 재라벨링
    feat_renamed = {
        col: (f"{col}{star_map.get(col,'')}" if col in star_map else col)
        for col in df_feat.columns
    }
    df_feat_renamed = df_feat.rename(columns=feat_renamed)

    # Dotplot
    df_melt = df_feat_renamed.T.reset_index().melt(
        id_vars="index", var_name="Cluster", value_name="Mean |SHAP|"
    )
    plt.figure(figsize=(10, 5))
    ax = sns.stripplot(
        x="Cluster",
        y="Mean |SHAP|",
        hue="index",
        data=df_melt,
        dodge=True,
        size=6,
        jitter=False,
        palette="tab10",
        alpha=0.9,
    )
    # 경계선 가이드
    cats = [t.get_text() for t in ax.get_xticklabels()]
    for i in range(len(cats) - 1):
        ax.axvline(i + 0.5, ls="--", lw=0.8, color="gray", alpha=0.5, zorder=0)
    ax.set_title(
        f"{outcome} — Cluster mean |SHAP| per Feature", fontsize=11, weight="bold"
    )
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Mean |SHAP|")
    plt.legend(title="Feature", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)
    plt.tight_layout()
    plt.savefig(
        FIGTAB_ROOT / f"Fig5A_{outcome}_dotplot_byFeature_guides_withStars.png", dpi=300
    )
    plt.close()

    # (B) Top N feature subplot: 제목에 별표 붙이기 (KW 기준)
    top_feats = df_feat.columns[:6]
    fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharey=True)
    for ax, feat in zip(axes.flat, top_feats):
        star = star_map.get(feat, "")
        sns.lineplot(x=df_feat.index, y=df_feat[feat], marker="o", color="black", ax=ax)
        ax.set_title(f"{feat}{star}", fontsize=10)
        ax.set_xlabel("Cluster")
        ax.set_ylabel("Mean |SHAP|")
        ax.set_xticklabels(ax.get_xticklabels(), ha="right")
    plt.suptitle(
        f"{outcome} — Cluster mean |SHAP| (Top 6 features)",
        fontsize=13,
        weight="bold",
    )
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.savefig(
        FIGTAB_ROOT / f"Fig5A_{outcome}_TopFeatures_clean_withStars.png", dpi=300
    )
    plt.close()

# (B) Δ(Top−Bottom) barplot & (C) Domain barplot은 기존 그림 유지
#    (원하면 bar label에 별표 추가 가능: df_all에서 원하는 조건으로 star를 조합)
# 그대로 복제해서 저장 위치만 figure_table로 변경:

# Δ(Top−Bottom) TopN
topN = 20
df_rank = df_all.sort_values("Δ(Top−Bottom)", ascending=False).head(topN)
plt.figure(figsize=(7.5, 6.5))
sns.barplot(
    y="Feature",
    x="Δ(Top−Bottom)",
    hue="Domain",
    data=df_rank,
    palette=palette,
    dodge=False,
)
plt.title(
    "Top features by SHAP heterogeneity (Δ Top−Bottom)", fontsize=12, weight="bold"
)
plt.xlabel("Δ (max−min mean |SHAP| across clusters)")
plt.ylabel("Feature")
plt.legend(title="Domain", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.savefig(FIGTAB_ROOT / "Fig5B_SHAP_Heterogeneity_Rank.png", dpi=300)
plt.close()

# Domain 별 평균 SD
df_dom = (
    df_all.groupby("Domain", as_index=False)["SD"]
    .mean()
    .sort_values("SD", ascending=False)
)
plt.figure(figsize=(6.5, 4.2))
sns.barplot(y="Domain", x="SD", data=df_dom, palette=palette)
plt.title("Average SHAP heterogeneity (SD) by Domain", fontsize=12, weight="bold")
plt.xlabel("Mean SD of mean |SHAP| across clusters")
plt.ylabel("")
plt.tight_layout()
plt.savefig(FIGTAB_ROOT / "Fig5C_Domain_SHAP_Heterogeneity.png", dpi=300)
plt.close()

print("✅ Outputs saved to:", FIGTAB_ROOT)

- ER, AD 제외

In [ ]:
FIGTAB_ROOT = RAWNORM_BASE / "cluster_figure_table/no_util"

In [ ]:
# ----- 유틸 -----
def stars_from_q(q):
    if pd.isna(q):
        return ""
    if q < 0.001:
        return "***"
    if q < 0.01:
        return "**"
    if q < 0.05:
        return "*"
    return ""


def parse_cluster_str(s):
    # "cluster_3" -> 3 / "C3" -> 3
    if pd.isna(s):
        return None
    m = re.search(r"(-?\d+)", str(s))
    return int(m.group(1)) if m else None


# ===== Domain 분류 함수 (기존과 동일) =====
# ===== Domain 분류 함수 =====
def infer_domain(feat: str) -> str:
    t = feat.lower()
    if any(k in t for k in ["aggression", "support", "social", "interpersonal", "llm"]):
        return "Aggression / Social function (LLM)"
    if any(k in t for k in ["er visit", "≥2 er", "≥3 er", "admission", "admit"]):
        return "Utilization"
    if any(
        k in t
        for k in [
            "bl3",
            "serum",
            "cholesterol",
            "hdl",
            "ldl",
            "triglyceride",
            "crp",
            "anc",
            "sodium",
            "na",
            "potassium",
            "k(",
            "hematocrit",
            "mchc",
            "eosinophil",
            "basophil",
            "alc",
            "lymphocyte",
        ]
    ):
        return "Lab"
    if any(
        k in t
        for k in [
            "mmpi",
            "hy",
            "mf",
            "vrin",
            "trinf",
            "psychometric",
            "audit",
            "hcl",
            "asrs",
            "ham",
        ]
    ):
        return "Psychometric"
    if any(
        k in t
        for k in [
            "olanz",
            "lith",
            "bzd",
            "quetiap",
            "antipsych",
            "treatment",
            "medic",
            "benzodiazepine",
        ]
    ):
        return "Treatment"
    if "sleep" in t:
        return "Sleep"
    if any(
        k in t
        for k in [
            "sex",
            "marital",
            "age",
            "education",
            "job",
            "drinking",
            "psychiatric family history",
            "hospitalization period",
        ]
    ):
        return "Demographic/Psychosocial"
    return "Other"


# ===== 1) 기존 Table4 요약 읽기 =====
f_table4 = RAWNORM_BASE / "Table4_CrossCluster_SHAP_Heterogeneity.xlsx"
df_all = pd.read_excel(f_table4, sheet_name="All_outcomes")
# 안전하게 Domain 재계산(혹시 비어있을 경우)
if "Domain" not in df_all.columns:
    df_all["Domain"] = df_all["Feature"].apply(infer_domain)

# ===== 2) 유의성 결과(각 label)와 결합 → 메인 테이블 확장 =====
LABELS = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]

# 결과 컬럼 초기화
for col in ["p_kw", "q_kw", "p_tb", "q_tb", "cliffs_delta", "sig_kw", "sig_tb"]:
    if col not in df_all.columns:
        df_all[col] = np.nan

# label별로 KW + Pairwise에서 Top-Bottom p를 주입
for lb in LABELS:
    sig_path = SIG_ROOT / f"{lb}__SHAP_cluster_significance_full.xlsx"
    if not sig_path.exists():
        print(f"[WARN] missing significance file: {sig_path}")
        continue
    df_kw = pd.read_excel(sig_path, sheet_name="Kruskal")
    df_pw = pd.read_excel(sig_path, sheet_name="Pairwise_All")

    # 매핑(dict) 준비
    qkw_map = dict(zip(df_kw["Feature"], df_kw["q_kw"]))
    pkw_map = dict(zip(df_kw["Feature"], df_kw["p_kw"]))

    # df_all 중 해당 outcome만 추림
    m_out = df_all["Outcome"] == lb
    sub = df_all.loc[m_out].copy()

    # 각 row에서 Top/Bottom 클러스터 번호 파싱 후 pairwise에서 해당 쌍의 p/q/효과크기 찾기
    p_tb_list, q_tb_list, d_tb_list, sig_tb_list = [], [], [], []
    p_kw_list, q_kw_list, sig_kw_list = [], [], []
    for _, row in sub.iterrows():
        feat = row["Feature"]
        # KW
        pkw = pkw_map.get(feat, np.nan)
        qkw = qkw_map.get(feat, np.nan)
        sig_kw = stars_from_q(qkw)

        # Top/Bottom 클러스터
        topc = parse_cluster_str(row.get("Max cluster") or row.get("Top cluster") or "")
        botc = parse_cluster_str(
            row.get("Min cluster") or row.get("Bottom cluster") or ""
        )

        # pairwise에서 두 방향(Cluster1,Cluster2) 모두 확인
        df_feat = df_pw[df_pw["Feature"] == feat]
        hit = df_feat[
            ((df_feat["Cluster1"] == topc) & (df_feat["Cluster2"] == botc))
            | ((df_feat["Cluster1"] == botc) & (df_feat["Cluster2"] == topc))
        ]
        if len(hit):
            p_tb = float(hit["p_raw"].iloc[0])
            q_tb = float(hit["q_fdr"].iloc[0])
            d_tb = float(hit["cliffs_delta"].iloc[0])
            sig_tb = stars_from_q(q_tb)
        else:
            p_tb = q_tb = d_tb = np.nan
            sig_tb = ""

        p_kw_list.append(pkw)
        q_kw_list.append(qkw)
        sig_kw_list.append(sig_kw)
        p_tb_list.append(p_tb)
        q_tb_list.append(q_tb)
        d_tb_list.append(d_tb)
        sig_tb_list.append(sig_tb)

    # 주입
    df_all.loc[m_out, "p_kw"] = p_kw_list
    df_all.loc[m_out, "q_kw"] = q_kw_list
    df_all.loc[m_out, "sig_kw"] = sig_kw_list
    df_all.loc[m_out, "p_tb"] = p_tb_list
    df_all.loc[m_out, "q_tb"] = q_tb_list
    df_all.loc[m_out, "cliffs_delta"] = d_tb_list
    df_all.loc[m_out, "sig_tb"] = sig_tb_list

# 저장(메인 테이블)
out_table_main = FIGTAB_ROOT / "Table4_with_significance.xlsx"
with pd.ExcelWriter(out_table_main) as xw:
    df_all.to_excel(xw, sheet_name="All_outcomes", index=False)
    for lb, sub in df_all.groupby("Outcome"):
        sub.to_excel(xw, sheet_name=lb, index=False)
print("✅ Saved main table with significance →", out_table_main)

# ===== 3) 서플: 모든 쌍 pairwise를 그대로 outcome별로 CSV 저장 =====
for lb in LABELS:
    sig_path = SIG_ROOT / f"{lb}__SHAP_cluster_significance_full.xlsx"
    if sig_path.exists():
        df_pw = pd.read_excel(sig_path, sheet_name="Pairwise_All")
        df_pw.to_csv(FIGTAB_ROOT / f"Supp_Pairwise_All_{lb}.csv", index=False)

# ===== 4) 그림에 별표 추가 (KW 기준) =====
palette = sns.color_palette("Set2")

# (A) Feature별 Dotplot: 범례(Feature 이름)에 별표 붙이기 (q_kw 기준)
for outcome in LABELS:
    # 히트맵 수치 파일 로드
    fpath = RAWNORM_BASE / outcome / f"{outcome}__heatmap_values_mean__TOP10_RAW.csv"
    if not fpath.exists():
        print(f"[WARN] missing heatmap CSV: {fpath}")
        continue
    df_feat = pd.read_csv(fpath, index_col=0)

    # 🌟🌟🌟 이 위치에 필터링 코드 추가 🌟🌟🌟
    EXCLUDE_VARS = ["≥2 ER Visits", "≥3 Admissions"]
    cols_to_keep = [col for col in df_feat.columns if col not in EXCLUDE_VARS]
    df_feat = df_feat[cols_to_keep]
    # 🌟🌟🌟 필터링 코드 끝 🌟🌟🌟

    # C 라벨로 보기 좋게
    df_feat.index = [c.replace("cluster_", "C") for c in df_feat.index]

    # 해당 outcome의 q_kw 맵(Feature→별표)
    sub_main = df_all[df_all["Outcome"] == outcome]
    qkw_map = dict(zip(sub_main["Feature"], sub_main["q_kw"]))
    star_map = {f: stars_from_q(q) for f, q in qkw_map.items()}

    # legend용 라벨 교체를 위해 컬럼명을 "원래+별"로 임시 재라벨링
    feat_renamed = {
        col: (f"{col}{star_map.get(col,'')}" if col in star_map else col)
        for col in df_feat.columns
    }
    df_feat_renamed = df_feat.rename(columns=feat_renamed)

    # Dotplot
    df_melt = df_feat_renamed.T.reset_index().melt(
        id_vars="index", var_name="Cluster", value_name="Mean |SHAP|"
    )
    plt.figure(figsize=(10, 5))
    ax = sns.stripplot(
        x="Cluster",
        y="Mean |SHAP|",
        hue="index",
        data=df_melt,
        dodge=True,
        size=6,
        jitter=False,
        palette="tab10",
        alpha=0.9,
    )
    # 경계선 가이드
    cats = [t.get_text() for t in ax.get_xticklabels()]
    for i in range(len(cats) - 1):
        ax.axvline(i + 0.5, ls="--", lw=0.8, color="gray", alpha=0.5, zorder=0)
    ax.set_title(
        f"{outcome} — Cluster mean |SHAP| per Feature", fontsize=11, weight="bold"
    )
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Mean |SHAP|")
    plt.legend(title="Feature", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)
    plt.tight_layout()
    plt.savefig(
        FIGTAB_ROOT / f"Fig5A_{outcome}_dotplot_byFeature_guides_withStars.png", dpi=300
    )
    plt.close()

    # (B) Top N feature subplot: 제목에 별표 붙이기 (KW 기준)
    top_feats = df_feat.columns[:6]
    fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharey=True)
    for ax, feat in zip(axes.flat, top_feats):
        star = star_map.get(feat, "")
        sns.lineplot(x=df_feat.index, y=df_feat[feat], marker="o", color="black", ax=ax)
        ax.set_title(f"{feat}{star}", fontsize=10)
        ax.set_xlabel("Cluster")
        ax.set_ylabel("Mean |SHAP|")
        ax.set_xticklabels(ax.get_xticklabels(), ha="right")
    plt.suptitle(
        f"{outcome} — Cluster mean |SHAP| (Top 6 features)",
        fontsize=13,
        weight="bold",
    )
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.savefig(
        FIGTAB_ROOT / f"Fig5A_{outcome}_TopFeatures_clean_withStars.png", dpi=300
    )
    plt.close()

# (B) Δ(Top−Bottom) barplot & (C) Domain barplot은 기존 그림 유지
#    (원하면 bar label에 별표 추가 가능: df_all에서 원하는 조건으로 star를 조합)
# 그대로 복제해서 저장 위치만 figure_table로 변경:

# Δ(Top−Bottom) TopN

# 🌟🌟🌟 이 위치에 필터링 코드 추가 🌟🌟🌟
EXCLUDE_VARS = ["AD_more_three", "ER_more_two"]
df_all_filtered = df_all[~df_all["Feature"].isin(EXCLUDE_VARS)].copy()
# 🌟🌟🌟 필터링 코드 끝 🌟🌟🌟

topN = 20
# df_rank = df_all.sort_values("Δ(Top−Bottom)", ascending=False).head(topN)
df_rank = df_all_filtered.sort_values("Δ(Top−Bottom)", ascending=False).head(topN)
plt.figure(figsize=(7.5, 6.5))
sns.barplot(
    y="Feature",
    x="Δ(Top−Bottom)",
    hue="Domain",
    data=df_rank,
    palette=palette,
    dodge=False,
)
plt.title(
    "Top features by SHAP heterogeneity (Δ Top−Bottom)", fontsize=12, weight="bold"
)
plt.xlabel("Δ (max−min mean |SHAP| across clusters)")
plt.ylabel("Feature")
plt.legend(title="Domain", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.savefig(FIGTAB_ROOT / "Fig5B_SHAP_Heterogeneity_Rank.png", dpi=300)
plt.close()

# Domain 별 평균 SD
# df_dom = (
#     df_all.groupby("Domain", as_index=False)["SD"]
#     .mean()
#     .sort_values("SD", ascending=False)
# )
df_all_viz = df_all[~df_all["Feature"].isin(EXCLUDE_VARS)]  # 🌟 필터링된 데이터 사용

df_dom = (
    df_all_viz.groupby("Domain", as_index=False)["SD"]  # 🌟 df_all_viz 사용
    .mean()
    .sort_values("SD", ascending=False)
)
plt.figure(figsize=(6.5, 4.2))
sns.barplot(y="Domain", x="SD", data=df_dom, palette=palette)
plt.title("Average SHAP heterogeneity (SD) by Domain", fontsize=12, weight="bold")
plt.xlabel("Mean SD of mean |SHAP| across clusters")
plt.ylabel("")
plt.tight_layout()
plt.savefig(FIGTAB_ROOT / "Fig5C_Domain_SHAP_Heterogeneity.png", dpi=300)
plt.close()

print("✅ Outputs saved to:", FIGTAB_ROOT)